## Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import os
import pyarrow
import openpyxl
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)



In [ ]:
# Define variables
today = datetime.now().date() 
delta = timedelta(days=13)
from_date = today - delta

#- delta
from_date = int(from_date.strftime('%Y%m%d'))


delta_13 = timedelta(days=13)
from_date_13 = today - delta
from_date_13 = int(from_date_13.strftime('%Y%m%d'))

delta_7 = timedelta(days=7)
from_date_7 = today - delta
from_date_7 = int(from_date_7.strftime('%Y%m%d'))

lostsale = 2
limit = 1.5
limit_of_lostsales = 0.03
from_date


## Define functions

In [3]:
#Đọc thư mục chứa file feather co gio
def Read_feather_folder1(path, date_from = 0):
    list_file = os.listdir(path)
    new_list = []
    for i in range(len(list_file)):
        if int(list_file[i][-18:-10]) >= date_from:
            new_list.append(list_file[i])
        
    df = pd.DataFrame()
    for file in new_list:
        file_path = os.path.join(path, file)
        df_temp = pd.read_feather(file_path)
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [4]:
#Đọc thư mục chứa file feather co ngay
def Read_feather_folder2(path, date_from = 0):
    list_file = os.listdir(path)
    new_list = []
    for i in range(len(list_file)):
        if int(list_file[i][-16:-8]) >= date_from:
            new_list.append(list_file[i])
        
    df = pd.DataFrame()
    for file in new_list:
        file_path = os.path.join(path, file)
        df_temp = pd.read_feather(file_path)
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [5]:
#Đọc thư mục chứa file feather
def Read_feather_folder(path):
    list_file = os.listdir(path)
    df = pd.DataFrame()
    for file in list_file:
        file_path = os.path.join(path, file)
        df_temp = pd.read_feather(file_path)
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

## Process data

### Dim data

In [6]:
store_hub = pd.read_excel(r'D:\Lostsale ĐM\DIM\hub_dongmat.xlsx', sheet_name='dim')
group = pd.read_excel(r'D:\Lostsale ĐM\DIM\ProductList.xlsx', sheet_name='group')

### Fact data

#### Stock 587

In [7]:
#Load dữ liệu 587
df_587 = Read_feather_folder1(r'D:\Lostsale ĐM\587\587_ft', date_from= from_date)

df_587['date_time'] = df_587['file_name'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

df_587[df_587['Trạng thái kinh doanh của sản phẩm'] == 'Kinh doanh bình thường']
#Xử lý cột date_time
df_587['time'] =df_587['date_time'].astype('str').str[8:].astype('int64')
df_587['date'] =df_587['date_time'].astype('str').str[:8].astype('int64')
df_587['date'] = pd.to_datetime(df_587['date'], format='%Y%m%d')
df_587.sort_values(by=[ 'Mã siêu thị', 'Mã Model','date', 'time'], ascending=[False, False, False, False], inplace=True)
df_587.drop_duplicates(subset=['Mã siêu thị', 'Mã Model','date'], inplace=True)
df_587.drop(columns=['file_name', 'date_time', 'Phân loại'], inplace=True)
df_587['Sức bán'] = df_587.groupby(['Mã siêu thị', 'Mã Model'])['Sức bán'].ffill()
df_587['Sức bán'] = df_587['Sức bán'].fillna(0)

#### Merge stock 587

In [ ]:
#Facing

df_587['month'] = df_587['date'].dt.month
facing = Read_feather_folder2(r'D:\Lostsale ĐM\Facing\facing_ft',  date_from= from_date)
facing['date'] =facing['file_name'].astype('str').str[-8:].astype('int64')
facing['date'] = pd.to_datetime(facing['date'], format='%Y%m%d')
facing.drop_duplicates(inplace=True)

facing=facing[(facing['Tổng số lượng'] > 0)][['Mã siêu thị', 'Mã Model', 'date']].drop_duplicates()
facing['Có vị trí trưng bày'] = 1
df_587_facing = df_587.merge(facing, how='left', on=['Mã siêu thị', 'Mã Model', 'date'])
df_587_facing['Có vị trí trưng bày'] = df_587_facing['Có vị trí trưng bày'].fillna(0)

#Label
label = Read_feather_folder(r'D:\Lostsale ĐM\ABC\ABC_ft_daily')
label['month'] = label['file_name'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
label = label[label['Loại hình phân loại'].isin(['PHÂN LOẠI SẢN PHẨM HÀNG ĐÔNG THEO SIÊU THỊ','PHÂN LOẠI SẢN PHẨM HÀNG KEM THEO SIÊU THỊ','PHÂN LOẠI SẢN PHẨM MÁT MẶN THEO SIÊU THỊ','PHÂN LOẠI SẢN PHẨM MÁT NGỌT THEO SIÊU THỊ'])]
label = label[['Mã siêu thị', 'Mã Model', 'Phân loại', 'month']].drop_duplicates()

df_587_facing_label = df_587_facing.merge(label, how='left', on=['Mã siêu thị', 'Mã Model', 'month'])
df_587_facing_label['Phân loại'] = df_587_facing_label['Phân loại'].fillna('C')

# Dim

df = df_587_facing_label.merge(store_hub, on='Mã siêu thị', how='left').merge(group, on='Nhóm hàng', how='left')
df.head()

In [ ]:
# Check duplicates
df.groupby(['date','time','Mã siêu thị', 'Mã sản phẩm cơ sở'])['Phân loại'].count().sort_values(ascending=False)


In [11]:
outstock = Read_feather_folder(r'D:\Lostsale ĐM\outstock\outstock ft')
outstock.drop_duplicates(inplace=True)
outstock['Số ngày đứt hàng'] =   (outstock['Ngày đứt hàng đến'] -outstock['Ngày đứt hàng từ']).dt.days
outstock['Ngày đứt hàng đến plus 7'] = outstock['Ngày đứt hàng đến'] + timedelta(days=7)
outstock_processed = outstock[outstock['Số ngày đứt hàng'] > 3][['Mã sản phẩm', 'Mã chi nhánh', 'Mã khu vực', 'Ngày đứt hàng từ', 'Ngày đứt hàng đến']].drop_duplicates()


outstock_processed['date'] = outstock_processed.apply(lambda row: pd.date_range(start=row['Ngày đứt hàng từ'], end=row['Ngày đứt hàng đến'], freq='D'), axis=1)

# Sử dụng phương pháp "exploding" để tạo các hàng mới cho mỗi ngày
outstock_processed = outstock_processed.explode('date')

outstock_processed = outstock_processed[['Mã sản phẩm', 'date', 'Mã khu vực']].drop_duplicates()
outstock_processed['đứt hàng'] = 1
outstock_processed.rename(columns={'Mã sản phẩm':'Mã sản phẩm cơ sở','Mã khu vực': 'Mã khu vực mua hàng C2'}, inplace=True)

In [12]:

# is lostsale
df['is_lostsales'] = 0
df.loc[(df['Tồn thực tế ở ST'] < lostsale), 'is_lostsales'] = 1

In [13]:
df = df[(df['date']< np.datetime64(today))].merge(df[(df['date']== np.datetime64(today)) & (df['is_lostsales']== 1) & (df['Có vị trí trưng bày']== 1)][['Mã siêu thị', 'Mã Model']].drop_duplicates(), how='right', on=['Mã siêu thị', 'Mã Model'])


#### Phiếu nhập

In [ ]:


im = Read_feather_folder2(r'D:\Lostsale ĐM\PN\PN ft', date_from= from_date_7)

im['Mã phiếu nhập'] = im['Mã phiếu nhập'].astype('str')
im['Mã sản phẩm'] = im['Mã sản phẩm'].astype('str')

im.drop(columns='file_name', inplace=True)
im['date'] = pd.to_datetime(im['Ngày nhập']).dt.date
im['date'] = pd.to_datetime(im['date'])

im = im.merge(slqd, how='left', on='Mã sản phẩm')
im['Số lượng đã quy đổi'] = im['Số lượng'] * im['Số lượng quy đổi']

im.head()


#### Phiếu xuất

In [16]:

out = Read_feather_folder2(r'D:\Lostsale ĐM\PX\PX ft',date_from= from_date_7)
out = out[['Mã siêu thị', 'Mã phiếu xuất', 'Mã hình thức xuất', 'Ngày xuất', 'Mã sản phẩm', 'Mã Model', 'Số lượng', 'totalbf', 'Giá bán', 'Mã cận date', 'Tên hình thức xuất bán']]
out['Mã phiếu xuất'] = out['Mã phiếu xuất'].astype('str')
out['Mã sản phẩm'] = out['Mã sản phẩm'].astype('str')

# out.drop(columns='file_name', inplace=True)
out['date'] = pd.to_datetime(out['Ngày xuất']).dt.date
out['date'] = pd.to_datetime(out['date'])

out = out.merge(slqd, how='left', on='Mã sản phẩm')
out['Số lượng đã quy đổi'] = out['Số lượng'] * out['Số lượng quy đổi']

pd.set_option('display.max_columns', None)
out.head()

,Mã siêu thị,Mã phiếu xuất,Mã hình thức xuất,Ngày xuất,Mã sản phẩm,Mã Model,Số lượng,totalbf,Giá bán,Mã cận date,Tên hình thức xuất bán,date,Số lượng quy đổi,Số lượng đã quy đổi
0,1696,23111016960314767,3,2023-11-07 05:48:55.182,1012980000059,1711001284,2,68600.0,34300.0,0,Xuất bán hàng tại siêu thị,2023-11-07,4.0,8.0
1,1970,23110019700425088,3,2023-11-07 06:08:42.119,8938502525016,1607003669,1,22000.0,22000.0,0,Xuất bán hàng tại siêu thị,2023-11-07,1.0,1.0
2,1970,23110019700425088,3,2023-11-07 06:08:42.119,9904639000002,2104000435,3,33000.0,11000.0,0,Xuất bán hàng tại siêu thị,2023-11-07,1.0,3.0
3,2012,23111020120152589,3,2023-11-07 06:09:14.980,9903929000096,2203000543,1,44000.0,44000.0,0,Xuất bán hàng tại siêu thị,2023-11-07,1.0,1.0
4,1970,23110019700425089,3,2023-11-07 06:14:58.523,9903929000096,2203000543,1,44000.0,44000.0,0,Xuất bán hàng tại siêu thị,2023-11-07,1.0,1.0


#### Concat

In [17]:

df_stripped = df[df['date']>=(pd.to_datetime(today) - timedelta(days=7))][['date', 'Mã Model', 'Mã siêu thị']]

df_im = df_stripped.merge(im, on=['date', 'Mã siêu thị', 'Mã Model'], how='left')
df_im.rename(columns={'Mã sản phẩm': 'Mã sản phẩm im', 'Số lượng đã quy đổi': 'Số lượng im'}, inplace=True)


df_im_g = df_im.groupby(['date', 'Mã siêu thị', 'Mã Model', 'Tên hình thức nhập'])['Số lượng im'].sum().reset_index()
df_im_pivot = df_im_g.pivot(index=['date', 'Mã Model', 'Mã siêu thị'], columns='Tên hình thức nhập', values=['Số lượng im']).fillna(0).reset_index()

df_out = df_stripped.merge(out, on=['date', 'Mã siêu thị', 'Mã Model'], how='left')
df_out = df_out.merge(slqd, how='left', on='Mã sản phẩm')
df_out.rename(columns={'Mã sản phẩm': 'Mã sản phẩm out', 'Số lượng đã quy đổi': 'Số lượng out'}, inplace=True)

df_out_g = df_out.groupby(['date', 'Mã siêu thị', 'Mã Model', 'Tên hình thức xuất bán'])['Số lượng out'].sum().reset_index()
df_out_pivot = df_out_g.pivot(index=['date', 'Mã Model', 'Mã siêu thị'], columns='Tên hình thức xuất bán', values=['Số lượng out']).fillna(0).reset_index()

df_all = df[df['date']>=(pd.to_datetime(today) - timedelta(days=7))].merge(df_im_pivot, how='left', on=['date', 'Mã siêu thị', 'Mã Model'])
df_all = df_all.merge(df_out_pivot, how='left', on=['date', 'Mã siêu thị', 'Mã Model'])
df_all.head()

,Mã siêu thị,Tên siêu thị,Mã loại kho,Loại kho,Mã Model,Mã sản phẩm cơ sở,Tên sp cơ sở,Đơn vị,Ngành hàng,Nhóm hàng,Trạng thái kinh doanh của sản phẩm,Tồn thực tế ở ST,Số lượng đang chuyển về từ kho khác,Lệnh chia còn hạn,Quy cách mua,Quy cách chia,SL tồn kho tối thiểu,Sức bán,Số ngày bán,Số ngày tồn kho tiêu chuẩn,Trọng lượng,Thể tích,Mã nhóm SP tính doanh thu,Ngành hàng - Phân tích,time,date,month,Có vị trí trưng bày,Phân loại,RSM,AM,Miền,Tỉnh/thành,Quận/huyện,Mô hình,TRẠNG THÁI,Tên DC,Mã DC,Thử nghiệm,Nhóm,is_lostsales,"(Số lượng im, Nhập chuyển kho)","(Số lượng im, Nhập mua nội bộ)","(Số lượng im, Nhập thừa so kiểm kê với sổ sách)","(Số lượng im, Nhập trả hàng BHX Online Pick tại Siêu Thị)","(Số lượng im, Nhập trả lại của hàng bán tại siêu thị)","(Số lượng im, Nhập trả lại của hàng xuất khuyến mãi)","(Số lượng out, Xuất bán hàng BHX Online Pick tại Siêu Thị)","(Số lượng out, Xuất bán hàng tại siêu thị)","(Số lượng out, Xuất bán nội bộ)","(Số lượng out, Xuất chuyển kho)","(Số lượng out, Xuất hủy)","(Số lượng out, Xuất khuyến mãi)","(Số lượng out, Xuất thiếu so kiểm kê với sổ sách)"
0,27921,BHX_HCM_GVA - Cityland Park Hills,1.0,Siêu thị,2301000763,9.904639e+12,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,Chai,Sản Phẩm Từ Sữa - Bảo Quản Mát,Sữa Các Loại,Kinh doanh bình thường,7.0,0.0,0.0,6.0,6.0,9.0,0.0,700.0,8.0,0.292,0.000484,2226.0,Mát ngọt,14.0,2023-11-14,11.0,1.0,C,5678 - Thị Ngân,2094 - Thùy Trinh,HCM,Quận Gò Vấp,Quận Gò Vấp,200m2,ON,BHX_HCM_Q12 - Kho DC Mini Đông Mát 63 Trần Thị Do,22859.0,0.0,MÁT NGỌT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,0.0,1.0,0.0
1,27921,BHX_HCM_GVA - Cityland Park Hills,1.0,Siêu thị,2301000763,9.904639e+12,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,Chai,Sản Phẩm Từ Sữa - Bảo Quản Mát,Sữa Các Loại,Kinh doanh bình thường,8.0,0.0,0.0,6.0,6.0,9.0,0.0,800.0,8.0,0.292,0.000484,2226.0,Mát ngọt,14.0,2023-11-13,11.0,1.0,C,5678 - Thị Ngân,2094 - Thùy Trinh,HCM,Quận Gò Vấp,Quận Gò Vấp,200m2,ON,BHX_HCM_Q12 - Kho DC Mini Đông Mát 63 Trần Thị Do,22859.0,0.0,MÁT NGỌT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,0.0,0.0,0.0,0.0,0.0
2,27921,BHX_HCM_GVA - Cityland Park Hills,1.0,Siêu thị,2301000763,9.904639e+12,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,Chai,Sản Phẩm Từ Sữa - Bảo Quản Mát,Sữa Các Loại,Kinh doanh bình thường,11.0,0.0,0.0,6.0,6.0,9.0,0.0,1100.0,8.0,0.292,0.000484,2226.0,Mát ngọt,0.0,2023-11-12,11.0,1.0,C,5678 - Thị Ngân,2094 - Thùy Trinh,HCM,Quận Gò Vấp,Quận Gò Vấp,200m2,ON,BHX_HCM_Q12 - Kho DC Mini Đông Mát 63 Trần Thị Do,22859.0,0.0,MÁT NGỌT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,27921,BHX_HCM_GVA - Cityland Park Hills,1.0,Siêu thị,2301000763,9.904639e+12,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,Chai,Sản Phẩm Từ Sữa - Bảo Quản Mát,Sữa Các Loại,Kinh doanh bình thường,11.0,0.0,0.0,6.0,6.0,9.0,0.0,1100.0,8.0,0.292,0.000484,2226.0,Mát ngọt,8.0,2023-11-11,11.0,1.0,C,5678 - Thị Ngân,2094 - Thùy Trinh,HCM,Quận Gò Vấp,Quận Gò Vấp,200m2,ON,BHX_HCM_Q12 - Kho DC Mini Đông Mát 63 Trần Thị Do,22859.0,0.0,MÁT NGỌT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27921,BHX_HCM_GVA - Cityland Park Hills,1.0,Siêu thị,2301000763,9.904639e+12,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,Chai,Sản Phẩm Từ Sữa - Bảo Quản Mát,Sữa Các Loại,Kinh doanh bình thường,11.0,0.0,0.0,6.0,6.0,9.0,0.0,1100.0,8.0,0.292,0.000484,2226.0,Mát ngọt,0.0,2023-11-10,11.0,1.0,C,5678 - Thị Ngân,2094 - Thùy Trinh,HCM,Quận Gò Vấp,Quận Gò Vấp,200m2,ON,BHX_HCM_Q12 - Kho DC Mini Đông Mát 63 Trần Thị Do,22859.0,0.0,MÁT NGỌT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Candate

In [18]:

cd = Read_feather_folder2(r'D:\Lostsale ĐM\Candate\Candate ft',date_from= from_date_7)
cd.drop_duplicates(inplace=True)
cd.drop(columns='file_name', inplace=True)
cd['Mã sản phẩm'] = cd['Mã sản phẩm'].astype('str')

cd = cd.merge(slqd, how='left', on='Mã sản phẩm')
cd['Sl bán cận date'] = cd['Sl bán cận date'] * cd['Số lượng quy đổi']
cd['SL hủy'] = cd['SL hủy'] * cd['Số lượng quy đổi']
cd['Số lượng tặng do cận date/hư hỏng'] = cd['Số lượng tặng do cận date/hư hỏng'] * cd['Số lượng quy đổi']
cd['SL xuất km'] = cd['SL xuất km'] * cd['Số lượng quy đổi']
cd_g = cd.groupby(['Ngày', 'Mã siêu thị', 'Mã Model'])['Sl bán cận date', 'DT cận date', 'SL hủy', 'Giá trị hủy', 'Số lượng tặng do cận date/hư hỏng', 'Giá vốn', 'SL xuất km', 'Giá trị tặng'].sum().reset_index()
cd_g.rename(columns={'Ngày': 'date'}, inplace=True)

In [19]:
df_all_cd = df_all.merge(cd_g, how='left', on= ['date', 'Mã siêu thị', 'Mã Model'])
df_all_cd.columns = df_all_cd.columns.map(str)

#### Map Khu vực và nhu cầu mua

In [20]:

df554 = Read_feather_folder2(r'D:\Lostsale ĐM\554\554 ft', date_from= from_date_13)
df554.drop(columns='file_name', inplace=True)
df554['Mã khu vực mua hàng'] = df554['Mã khu vực mua hàng'].astype('str')
df554['date'] = pd.to_datetime(df554['Ngày cập nhật'], unit='D', origin=pd.Timestamp('1899-12-30')).dt.date

df554['date'] = pd.to_datetime(df554['date'] )

df554['leadtime'] = df554['jsondata'].str.extract(r'"LEADTIME":(\d+)')
df554['fcfromdate'] = df554['jsondata'].str.extract(r'"FCFROMDATE\\":(\d+)')
df554['fctodate'] = df554['jsondata'].str.extract(r'"FCTODATE\\":(\d+)')
df554['CAPACITYRAW'] = df554['jsondata'].str.extract(r'"CAPACITYRAW":(\d+)')


df554.sort_values(by=['Mã Model', 'Mã siêu thị', 'date', 'Ngày cập nhật'], ascending=[True, True, True, False], inplace=True)
df554.drop_duplicates(subset=['date', 'Mã Model', 'Mã siêu thị', 'Mã khu vực mua hàng'], inplace=True)
df554.groupby(['date', 'Mã Model', 'Mã siêu thị'])['Ngày cập nhật'].count().sort_values(ascending=False)

date        Mã Model    Mã siêu thị
2023-11-04  2006000583  3112           2
2023-11-06  2107002941  5128           2
            2009000593  6466           2
2023-11-04  2308000969  2108           2
2023-11-06  2308000943  6841           2
                                      ..
            2107002944  6597           1
                        6580           1
                        6563           1
                        6562           1
2023-11-14  2310000948  10438          1
Name: Ngày cập nhật, Length: 1850999, dtype: int64

In [21]:



df911 = Read_feather_folder2(r'D:\Lostsale ĐM\911\911 ft', date_from= from_date_7)

df911.drop(columns='file_name', inplace=True)

df911['Ngày cập nhật'] = pd.to_datetime(df911['Ngày cập nhật'], unit='D', origin=pd.Timestamp('1899-12-30'))
df911['Ngày đặt'] = pd.to_datetime(df911['Ngày đặt'], unit='D', origin=pd.Timestamp('1899-12-30')).dt.date
df911['Ngày đặt'] = pd.to_datetime(df911['Ngày đặt']).dt.date
df911['Ngày nhận hàng'] = pd.to_datetime(df911['Ngày nhận hàng']).dt.date

df911['date'] = pd.to_datetime(df911['Ngày đặt']).dt.date
df911['date'] = pd.to_datetime(df911['date'])
df911['Khu vực mua hàng'] = df911['Khu vực mua hàng'].astype('str')
df911.sort_values(by=['Model', 'Tên kho nhận', 'Tên siêu thị', 'Khu vực mua hàng', 'date','Mã PO', 'Ngày cập nhật'], ascending=[True, True, True, True,True, True, False], inplace=True)
df911.drop_duplicates(subset=['date','Model', 'Tên kho nhận', 'Tên siêu thị', 'Khu vực mua hàng'], inplace=True)

df911['leadtime'] = df911['jsondata'].str.extract(r'"LEADTIME":(\d+)')
df911['fcfromdate'] = df911['jsondata'].str.extract(r'"FCFROMDATE\\":(\d+)')
df911['fctodate'] = df911['jsondata'].str.extract(r'"FCTODATE\\":(\d+)')


df911['Mã Model'] = df911['Model'].str.extract('(^[0-9]{0,10})').astype('int64')
df911['Mã siêu thị'] = df911['Tên siêu thị'].str.extract('(^[0-9]{0,10})').astype('int64')
df911['Mã kho nhận'] = df911['Tên kho nhận'].str.extract('(^[0-9]{0,10})').astype('int64')
df911['Kho nhận'] = df911['Tên kho nhận'].str.extract('-(.*)$')
df911['Mã khu vực mua hàng 911'] = df911['Khu vực mua hàng'].str.extract('(^[0-9]{0,10})').astype('int64')
df911['Khu vực mua hàng 911'] = df911['Khu vực mua hàng'].str.extract('-(.*)$')
df911.drop(columns=['Tên siêu thị', 'Model', 'Tên kho nhận', 'Khu vực mua hàng'], inplace=True)

df911['CAPACITYRAW'] = df911['jsondata'].str.extract(r'"CAPACITYRAW":(\d+)')
df911['CAPACITYRAW'] = df911['CAPACITYRAW'].fillna(df911['SL max ngăn tủ'])
df911['SL max ngăn tủ'] = df911['CAPACITYRAW']

In [ ]:

df = df[df['Mã loại kho'].isin([1,19])]
df_kvmh = df[['Mã siêu thị', 'Mã Model', 'Tên siêu thị', 'date']].drop_duplicates()
df_kvmh_554 = df554[['date', 'Mã siêu thị', 'Mã Model', 'Mã khu vực mua hàng', 'Khu vực mua hàng']].drop_duplicates()
df_kvmh_554.rename(columns={'Mã khu vực mua hàng': 'Mã khu vực mua hàng 554', 'Khu vực mua hàng': 'Khu vực mua hàng 554'}, inplace=True)
df_kvmh_991 = df911[['date', 'Mã siêu thị', 'Mã Model', 'Mã kho nhận', 'Mã khu vực mua hàng 911']].drop_duplicates()
df_kvmh_991.head()

In [23]:
df_kvmh_991['date'] = pd.to_datetime(df_kvmh_991['date'])
df_kvmh_911_merge = df_kvmh.merge(df_kvmh_991, how='left', on=['Mã siêu thị', 'Mã Model', 'date'])
df_kvmh_911_merge['RN_911'] = df_kvmh_911_merge.sort_values(['date', 'Mã siêu thị', 'Mã Model'], ascending=[False, False,False]) \
             .groupby(['date', 'Mã siêu thị', 'Mã Model']) \
             .cumcount() + 1
df_kvmh_911_merge['RN_911'].max()

1

In [24]:
df_kvmh_554['date'] = pd.to_datetime(df_kvmh_554['date'])
df_kvmh_554['RN_554'] = df_kvmh_554.sort_values(['date', 'Mã siêu thị', 'Mã Model'], ascending=[False, False,False]) \
             .groupby(['date', 'Mã siêu thị', 'Mã Model']) \
             .cumcount() + 1
df_kvmh_554['RN_554'].max()

2

In [ ]:
df_kvmh_911_merge_554 = df_kvmh_911_merge.merge(df_kvmh_554, how='left', on=['date', 'Mã siêu thị', 'Mã Model'])
df_kvmh_911_merge_554['Mã khu vực mua hàng 911'] =  df_kvmh_911_merge_554['Mã khu vực mua hàng 911'].fillna(0).astype('int64')
df_kvmh_911_merge_554['Mã khu vực mua hàng 554'] =  df_kvmh_911_merge_554['Mã khu vực mua hàng 554'].fillna(0).astype('int64')
df_kvmh_911_merge_554.head()

In [26]:
no_911_yes_554 = df_kvmh_911_merge_554[(df_kvmh_911_merge_554['Mã khu vực mua hàng 911'] == 0) & (df_kvmh_911_merge_554['Mã khu vực mua hàng 554'] != 0) ][['date', 'Mã Model', 'Mã khu vực mua hàng 554']].drop_duplicates()
no_911_yes_554_map911 = no_911_yes_554.merge(df_kvmh_991, how='left', right_on=['date', 'Mã Model', 'Mã khu vực mua hàng 911'], left_on=['date', 'Mã Model', 'Mã khu vực mua hàng 554'])
# phat sinh 2 siêu thi khi map qua  911

no_911_yes_554_map911['diff2'] = 0
no_911_yes_554_map911.loc[(no_911_yes_554_map911['Mã siêu thị'] - no_911_yes_554_map911['Mã kho nhận']) != 0, 'diff2'] = 1
no_911_yes_554_map911.sort_values(by=['date', 'Mã Model', 'Mã khu vực mua hàng 554', 'diff2'], ascending=[False, False, False, False], inplace=True)
no_911_yes_554_map911.drop_duplicates(subset=['date', 'Mã Model', 'Mã khu vực mua hàng 554'], inplace=True)


In [27]:
no_911_yes_554_map911_l2 = no_911_yes_554_map911.merge(df_kvmh_991, how='left', on=['date', 'Mã Model', 'Mã siêu thị'], suffixes=('', '_map_lan_2'))
no_911_yes_554_map911_l2['diff'] = 0
no_911_yes_554_map911_l2.loc[(no_911_yes_554_map911_l2['Mã khu vực mua hàng 911'].fillna(0).astype('int64') - no_911_yes_554_map911_l2['Mã khu vực mua hàng 911_map_lan_2'].fillna(0).astype('int64')) != 0, 'diff'] = 1
no_911_yes_554_map911_l2.sort_values(by=['date', 'Mã Model', 'Mã khu vực mua hàng 554', 'Mã siêu thị', 'diff'], ascending=[False, False, False, False, False], inplace=True)
no_911_yes_554_map911_l2.drop_duplicates(subset=['date', 'Mã Model', 'Mã khu vực mua hàng 554', 'Mã siêu thị'], inplace=True)
no_911_yes_554_map911_l2.head()

,date,Mã Model,Mã khu vực mua hàng 554,Mã siêu thị,Mã kho nhận,Mã khu vực mua hàng 911,diff2,Mã kho nhận_map_lan_2,Mã khu vực mua hàng 911_map_lan_2,diff
0,2023-11-14,2310000339,45212,NaN,NaN,NaN,1,NaN,NaN,0
1,2023-11-14,2310000339,45139,NaN,NaN,NaN,1,NaN,NaN,0
2,2023-11-14,2310000339,42910,NaN,NaN,NaN,1,NaN,NaN,0
3,2023-11-14,2310000339,30903,NaN,NaN,NaN,1,NaN,NaN,0
4,2023-11-14,2310000339,18031,NaN,NaN,NaN,1,NaN,NaN,0


In [28]:
df_kvmh_911_merge_554.rename(columns={'Mã siêu thị':'Mã siêu thị ori' ,'Mã kho nhận': 'Mã kho nhận ori 911', 'Kho nhận': 'Kho nhận ori 911', 'Mã khu vực mua hàng 911': 'Mã khu vực mua hàng 911 ori', 'Khu vực mua hàng 911': 'Khu vực mua hàng 911 ori'}, inplace=True)
df_kvmh_911_merge_554_911 = df_kvmh_911_merge_554.merge(no_911_yes_554_map911_l2, on=['date', 'Mã Model', 'Mã khu vực mua hàng 554'], how='left')
df_kvmh_911_merge_554_911.head()

,Mã siêu thị ori,Mã Model,Tên siêu thị,date,Mã kho nhận ori 911,Mã khu vực mua hàng 911 ori,RN_911,Mã khu vực mua hàng 554,Khu vực mua hàng 554,RN_554,Mã siêu thị,Mã kho nhận,Mã khu vực mua hàng 911,diff2,Mã kho nhận_map_lan_2,Mã khu vực mua hàng 911_map_lan_2,diff
0,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-14,NaN,0,1,27926,HÀNG MÁT - SỮA_DC MINI 63 TRẦN THỊ DO,1.0,NaN,NaN,NaN,1.0,NaN,NaN,0.0
1,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-13,NaN,0,1,27926,HÀNG MÁT - SỮA_DC MINI 63 TRẦN THỊ DO,1.0,NaN,NaN,NaN,1.0,NaN,NaN,0.0
2,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-12,NaN,0,1,27926,HÀNG MÁT - SỮA_DC MINI 63 TRẦN THỊ DO,1.0,22859.0,22859.0,27926.0,0.0,6320.0,23901.0,1.0
3,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-11,NaN,0,1,27926,HÀNG MÁT - SỮA_DC MINI 63 TRẦN THỊ DO,1.0,22859.0,22859.0,27926.0,0.0,6320.0,23901.0,1.0
4,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-10,NaN,0,1,27926,HÀNG MÁT - SỮA_DC MINI 63 TRẦN THỊ DO,1.0,22859.0,22859.0,27926.0,0.0,6320.0,23901.0,1.0


In [ ]:
# 911 ori có
ori911 = df_kvmh_911_merge_554_911[df_kvmh_911_merge_554_911['Mã khu vực mua hàng 911 ori'] != 0]
subset = ori911[['date', 'Mã Model', 'Mã kho nhận ori 911']].drop_duplicates()
subset = subset.merge(df_kvmh_991, how='left', left_on=['date', 'Mã Model', 'Mã kho nhận ori 911'], right_on=['date', 'Mã Model', 'Mã siêu thị'])
subset.groupby(['date', 'Mã Model', 'Mã kho nhận ori 911'])['Mã khu vực mua hàng 911'].count().sort_values(ascending=False)
subset.rename(columns={'Mã siêu thị': 'Mã hub nhận hàng 911_911', 'Mã kho nhận':'Mã kho nhận 911_911', 'Kho nhận' : 'Kho nhận 911_911', 'Mã khu vực mua hàng 911' :'Mã khu vực mua hàng 911 911_911', 'Khu vực mua hàng 911': 'Khu vực mua hàng 911 911_911'}, inplace=True)
ori911 = ori911.merge(subset, how='left', on=['date', 'Mã Model', 'Mã kho nhận ori 911'])
ori911.drop(columns=['Mã khu vực mua hàng 554', 'Khu vực mua hàng 554', 'RN_554',
       'Mã siêu thị', 'Mã kho nhận', 'Mã khu vực mua hàng 911', 'diff2', 'Mã kho nhận_map_lan_2', 'Mã khu vực mua hàng 911_map_lan_2', 'diff', 'Mã hub nhận hàng 911_911', 'RN_911'], inplace=True)
ori911.rename(columns={'Mã kho nhận ori 911': 'Mã kho nhận C1', 'Kho nhận ori 911': 'Kho nhận C1', 'Mã khu vực mua hàng 911 ori': 'Mã khu vực mua hàng C1', 'Khu vực mua hàng 911 ori': 'Khu vực mua hàng C1', 'Mã kho nhận 911_911': 'Mã kho nhận C2', 'Kho nhận 911_911': 'Kho nhận C2', 'Mã khu vực mua hàng 911 911_911': 'Mã khu vực mua hàng C2', 'Khu vực mua hàng 911 911_911': 'Khu vực mua hàng C2'}, inplace=True)
ori911['no_data'] = 0
ori911.head()

In [30]:
# 911 ori không có, 554 có
ori554 = df_kvmh_911_merge_554_911[(df_kvmh_911_merge_554_911['Mã khu vực mua hàng 911 ori'] == 0) & (df_kvmh_911_merge_554_911['Mã khu vực mua hàng 554'] != 0)]
ori554 = ori554[~ori554['Mã khu vực mua hàng 911'].isna()]


In [ ]:
# ori554.groupby(['date', 'Mã siêu thị ori', 'Mã Model'])['RN_554'].count().sort_values(ascending=False)
ori554.drop(columns=['Mã khu vực mua hàng 911 ori', 'RN_911', 'Mã khu vực mua hàng 554', 'Khu vực mua hàng 554', 'RN_554', 'Mã siêu thị', 'diff2', 'diff'], inplace=True)
ori554.rename(columns={'Mã kho nhận': 'Mã kho nhận C1', 'Kho nhận': 'Kho nhận C1', 'Mã khu vực mua hàng 911' : 'Mã khu vực mua hàng C1', 'Khu vực mua hàng 911': 'Khu vực mua hàng C1', 'Mã kho nhận_map_lan_2': 'Mã kho nhận C2','Kho nhận_map_lan_2': 'Kho nhận C2', 'Mã khu vực mua hàng 911_map_lan_2': 'Mã khu vực mua hàng C2', 'Khu vực mua hàng 911_map_lan_2': 'Khu vực mua hàng C2'}, inplace = True)
ori554['no_data'] = 0
ori554.head()

In [32]:
# cả 2 không có
no_all = df_kvmh_911_merge_554_911[(df_kvmh_911_merge_554_911['Mã khu vực mua hàng 911 ori'] == 0) & (df_kvmh_911_merge_554_911['Mã khu vực mua hàng 554'] == 0)][['date', 'Mã siêu thị ori', 'Mã Model', 'Tên siêu thị']]
no_all['no_data'] = 1

In [ ]:
# Concat and check
khu_vuc = pd.concat([ori911, ori554, no_all])
khu_vuc.drop_duplicates(inplace=True)
khu_vuc.rename(columns={'Mã siêu thị ori': 'Mã siêu thị'}, inplace=True)
df_khu_vuc = df_kvmh.merge(khu_vuc, how='left', on=['Mã Model', 'Mã siêu thị', 'Tên siêu thị', 'date'])
print(df_khu_vuc['no_data'].isna().sum(),khu_vuc.groupby(['date', 'Mã Model', 'Mã siêu thị', 'Tên siêu thị'])['no_data'].count().sort_values(ascending=False))

In [ ]:
df_khu_vuc['Mã kho nhận C2'] = df_khu_vuc['Mã kho nhận C2'].fillna(df_khu_vuc['Mã kho nhận C1'])
df_khu_vuc['Mã khu vực mua hàng C2'] = df_khu_vuc['Mã khu vực mua hàng C2'].fillna(df_khu_vuc['Mã khu vực mua hàng C1'])
msp = df[['Mã Model', 'Mã sản phẩm cơ sở']].drop_duplicates()
df_khu_vuc = df_khu_vuc.merge(msp, how='left', on='Mã Model')
df_khu_vuc.head()

In [35]:
df_khu_vuc['no_data'].isna().sum()
df_khu_vuc = df_khu_vuc[df_khu_vuc['no_data']==0]

In [36]:
df_khu_vuc['date'] = pd.to_datetime(df_khu_vuc['date'])
df554['date'] = pd.to_datetime(df554['date'])
df911['date'] = pd.to_datetime(df911['date'])

df_khu_vuc = df_khu_vuc.merge(df554[['date', 'Mã siêu thị', 'Mã Model', 'nhucautheosb', 'nhucaust', 'leadtime', 'fcfromdate', 'fctodate', 'CAPACITYRAW']], how='left', on=['date', 'Mã siêu thị', 'Mã Model'])

df_khu_vuc.rename(columns={'leadtime': 'leadtime554', 'fcfromdate': 'fcfromdate554', 'fctodate': 'fctodate554'}, inplace=True)

df_khu_vuc = df_khu_vuc.merge(df911[['date', 'Mã siêu thị', 'Mã Model','Ngày nhận hàng', 'Số lượng cầu ban đầu', 'Số lượng cần mua', 'leadtime', 'fcfromdate', 'fctodate', 'SL max ngăn tủ']], how='left', on=['date', 'Mã siêu thị', 'Mã Model'])

In [ ]:
df_khu_vuc['nhucautheosb'] = df_khu_vuc['nhucautheosb'].fillna(df_khu_vuc['Số lượng cầu ban đầu'])
df_khu_vuc['nhucaust'] = df_khu_vuc['nhucaust'].fillna(df_khu_vuc['Số lượng cần mua'])
df_khu_vuc['leadtime554'] = df_khu_vuc['leadtime554'].fillna(df_khu_vuc['leadtime'])
df_khu_vuc['fcfromdate554'] = df_khu_vuc['fcfromdate554'].fillna(df_khu_vuc['fcfromdate'])
df_khu_vuc['fctodate554'] = df_khu_vuc['fctodate554'].fillna(df_khu_vuc['fctodate'])
df_khu_vuc['CAPACITYRAW'] = df_khu_vuc['CAPACITYRAW'].fillna(df_khu_vuc['SL max ngăn tủ'])
df_khu_vuc['CAPACITYRAW'] = df_khu_vuc['CAPACITYRAW'].fillna(0).astype('int64')

df_khu_vuc.drop(columns=['Số lượng cầu ban đầu', 'Số lượng cần mua', 'leadtime', 'fcfromdate', 'fctodate', 'SL max ngăn tủ', 'Mã kho nhận ori 911'], inplace=True)
df_khu_vuc.rename(columns={'leadtime554': 'leadtime', 'fcfromdate554': 'fcfromdate', 'fctodate554': 'fctodate'}, inplace=True)

df_khu_vuc.head()

#### NLG, Lịch, Điểm NCC

In [38]:
NLG = Read_feather_folder(r'D:\Lostsale ĐM\NLG\NLG ft')

# Xử lý cột date_time
# NLG['date'] =NLG['file_name'].astype('str').str[20:28].astype('int64')
# NLG['date'] = pd.to_datetime(NLG['date'], format='%d%m%Y')


NLG.drop(columns=['file_name', 'Ghi chú', 'Ngày cập nhật', 'Nhân viên cập nhập'], inplace=True)



NLG.rename(columns={'Mã khu vực': 'Mã khu vực mua hàng'}, inplace=True)
NLG['Mã sản phẩm'] = NLG['Mã sản phẩm'].astype('int64')
NLG['Mã khu vực mua hàng'] = NLG['Mã khu vực mua hàng'].astype('str')




supplier = pd.read_excel(r'D:\Lostsale ĐM\CN NCC vs NCC.xlsx')
supplier = supplier[['Mã chi nhánh', 'Mã NCC']].drop_duplicates()
NLG = NLG.merge(supplier, how='left', on='Mã chi nhánh')
NLG[NLG['Mã NCC'].isna()]

,Mã sản phẩm,Tên sản phẩm,Mã chi nhánh,Tên chi nhánh NCC,Mã khu vực mua hàng,Tên khu vực,Mã loại khu vực mua hàng,Mã kho nhận hàng,Tên kho nhận hàng,Hình thức giao hàng,Mã kho nhận hàng quá cảnh,Tên kho nhận hàng quá cảnh,Có hóa đơn về cùng đơn hàng,Khả năng cung ứng,Khả năng giao tối thiểu theo khu vực,Mã NCC


In [ ]:
NCC = Read_feather_folder(r'D:\Lostsale ĐM\Điểm NCC\Điểm NCC ft')

# NCC['date'] =NCC['file_name'].astype('str').str[-15:-7]
# NCC['date'] = pd.to_datetime(NCC['date'], format='%Y%m%d')


NCC.drop(columns=['file_name', 'Tên sản phẩm', 'Nhóm hàng', 'nccname', 'Khu vực mua hàng', 'Khả năng giao NCC','Đơn vị', 'MOQ Sản phẩm', 'Giá chuẩn', 'Đơn giá tặng/bán kèm', 'Ngày cập nhật', 'Chiết khấu khóa mua hàng', 'potype', 'Tên hình thức nhập'], inplace=True)
NCC = NCC[~NCC['Mã khu vực mua hàng'].isna()]
NCC = NCC.drop_duplicates()

NCC['Mã sản phẩm'] = NCC['Mã sản phẩm'].astype('int64')

NCC['Mã khu vực mua hàng'] = NCC['Mã khu vực mua hàng'].astype('int64').astype('str')
NCC['Mã NCC'] = NCC['Mã NCC'].astype('int64')

NCC.head()

In [ ]:
schedule = Read_feather_folder(r'D:\Lostsale ĐM\Lịch PO\Lịch PO ft')

# schedule['date'] =schedule['file_name'].astype('str').str[-12:-4]
# schedule['date'] = pd.to_datetime(schedule['date'], format='%d%m%Y')

schedule = schedule[['Mã chi nhánh NCC', 'Mã KV mua hàng', 'Lịch tạo PO theo thứ (thường)', 'Tên nhóm hàng']].drop_duplicates()
schedule.rename(columns={'Mã chi nhánh NCC': 'Mã chi nhánh', 'Mã KV mua hàng': 'Mã khu vực mua hàng'}, inplace=True)
schedule['Mã khu vực mua hàng'] = schedule['Mã khu vực mua hàng'].astype('str')
schedule.head()




In [41]:
schedule_expand = pd.DataFrame(columns=schedule.columns)


# Chuyển dữ liệu từ Table 1 sang Table 2 với các giá trị trong cột 'Lịch tạo PO theo thứ (thường)' được chia thành các giá trị riêng biệt
for _, row in schedule.iterrows():
    values = row['Lịch tạo PO theo thứ (thường)'].split(', ')
    for value in values:
        new_row = row.copy()
        new_row['Lịch tạo PO theo thứ (thường)'] = value
        schedule_expand = schedule_expand.append(new_row, ignore_index=True)

# Hiển thị Table 2
schedule_expand.head()


In [42]:

wd = {'Lịch tạo PO theo thứ (thường)': ['T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'CN'],
        'weekday': [0, 1, 2, 3, 4, 5, 6]}

df_wd = pd.DataFrame(wd)
schedule_expand = schedule_expand.merge(df_wd, how='left', on=['Lịch tạo PO theo thứ (thường)'])
schedule_expand.drop(columns=['Lịch tạo PO theo thứ (thường)'], inplace=True)
schedule_expand.head()


In [ ]:
NLG.rename(columns={'Mã sản phẩm': 'Mã sản phẩm cơ sở', 'Mã khu vực mua hàng': 'Mã khu vực mua hàng C2'}, inplace=True)
NLG['Mã khu vực mua hàng C2'] = NLG['Mã khu vực mua hàng C2'].astype('int64')
df_khu_vuc = df_khu_vuc.merge(NLG, how='left', on=['Mã sản phẩm cơ sở', 'Mã khu vực mua hàng C2'])


NCC.rename(columns={'Mã sản phẩm': 'Mã sản phẩm cơ sở', 'Mã khu vực mua hàng': 'Mã khu vực mua hàng C2'}, inplace=True)
NCC['Mã khu vực mua hàng C2'] = NCC['Mã khu vực mua hàng C2'].astype('int64')
df_khu_vuc = df_khu_vuc.merge(NCC, how='left', on =['Mã sản phẩm cơ sở', 'Mã khu vực mua hàng C2', 'Mã NCC'])

df = df[df['Mã loại kho'].isin([1,19])]
df_nhomhang = df[['Mã sản phẩm cơ sở', 'Nhóm hàng']].drop_duplicates()
df_nhomhang.rename(columns={'Nhóm hàng': 'Tên nhóm hàng'}, inplace=True)
df_khu_vuc = df_khu_vuc.merge(df_nhomhang, how='left', on='Mã sản phẩm cơ sở')

schedule.rename(columns={'Mã khu vực mua hàng': 'Mã khu vực mua hàng C2'}, inplace=True)
schedule['Mã khu vực mua hàng C2'] = schedule['Mã khu vực mua hàng C2'].astype('int64')
df_khu_vuc = df_khu_vuc.merge(schedule, how='left', on=['Mã chi nhánh', 'Mã khu vực mua hàng C2','Tên nhóm hàng'])


df_khu_vuc['weekday'] = df_khu_vuc['date'].dt.weekday
schedule_expand.rename(columns={'Mã khu vực mua hàng': 'Mã khu vực mua hàng C2'}, inplace=True)
schedule_expand['có lịch'] = 1
schedule_expand['Mã khu vực mua hàng C2'] = schedule_expand['Mã khu vực mua hàng C2'].astype('int64')
df_khu_vuc = df_khu_vuc.merge(schedule_expand, how='left', on=['Mã chi nhánh', 'Mã khu vực mua hàng C2','Tên nhóm hàng', 'weekday'])
df_khu_vuc.head()



In [46]:
df_khu_vuc['Khả năng cung ứng'] = df_khu_vuc['Khả năng cung ứng'].fillna(0)
df_khu_vuc['Lịch tạo PO theo thứ (thường)'] = df_khu_vuc['Lịch tạo PO theo thứ (thường)'].fillna(0).astype('str')
df_khu_vuc['có lịch'] = df_khu_vuc['có lịch'].fillna(0)

data_2 = df_khu_vuc[(df_khu_vuc['no_data'] == 0) & (df_khu_vuc['Khả năng cung ứng'] > 0) & (df_khu_vuc['Lịch tạo PO theo thứ (thường)'] != '0')]

data_1 = pd.concat([df_khu_vuc[(df_khu_vuc['no_data'] == 1) | (df_khu_vuc['Khả năng cung ứng'] == 0) | (df_khu_vuc['Lịch tạo PO theo thứ (thường)'] == '0')], no_all])


data_2['Mode'] = 0
hub = pd.read_excel(r'D:\Lostsale ĐM\DIM\hub_dongmat.xlsx', sheet_name='DC')
data_2 = data_2.merge(hub, how='left', on='Mã kho nhận C1')
data_2['Mã kho nhận C1 là hub'] = data_2['Mã kho nhận C1 là hub'].fillna(0)
data_2.loc[((data_2['Mã kho nhận C1'] == data_2['Mã kho nhận C2']) & (data_2['Mã kho nhận C1'] == data_2['Mã siêu thị'])), 'Mode'] = 1
data_2.loc[((data_2['Mã kho nhận C1'] == data_2['Mã kho nhận C2']) & (data_2['Mã kho nhận C1'] != data_2['Mã siêu thị']) & (data_2['Mã kho nhận C1 là hub'] == 1)), 'Mode'] = 2
data_2.loc[((data_2['Mã kho nhận C1'] == data_2['Mã kho nhận C2']) & (data_2['Mã kho nhận C1'] != data_2['Mã siêu thị']) & (data_2['Mã kho nhận C1 là hub'] == 0)), 'Mode'] = 3

In [47]:
data_2['fcfromdate'] = data_2['fcfromdate'].fillna(data_2['Ngày nhận hàng'])
# data_2['fctodate'] = data_2['fctodate'].fillna(data_2['Ngày nhận hàng']+pd.to_timedelta(data_2['Số ngày tồn kho tiêu chuẩn'], unit='D'))
data_2['fcfromdate'] = pd.to_datetime(data_2['fcfromdate'])
# data_2['fctodate'] = pd.to_datetime(data_2['fctodate'])
data_2['fcfromdate +1'] = data_2['fcfromdate'] + timedelta(days=1)

In [48]:
df911_nhucau = df911[['date', 'Mã Model', 'Mã siêu thị','Mã kho nhận', 'Mã khu vực mua hàng 911', 'Số lượng cầu ban đầu', 'Số lượng cần mua','Số lượng đang đặt', 'Số lượng đặt về kho trung tâm', 'Tồn kho', 'Tồn kho trung tâm', 'Số lượng sc được nhận', 'Số lượng sc đã chuyển đi','SL mapping', 'Mã PO', 'Số lượng hàng đã đặt', 'Ghi chú vi phạm đơn hàng', 'Ghi chú vi phạm sản phẩm', 'SL vi phạm thể tích']].drop_duplicates()
df911_nhucau.rename(columns={'Mã kho nhận': 'Mã kho nhận C2', 'Mã khu vực mua hàng 911': 'Mã khu vực mua hàng C2', 'Mã siêu thị': 'Mã siêu thị map nhu cau 911'}, inplace=True)

In [ ]:

data_2['Mã kho nhận C1'] = data_2['Mã kho nhận C1'].astype('int64')
data_2['Mã kho nhận C2'] = data_2['Mã kho nhận C2'].astype('int64')
data_2['Mã khu vực mua hàng C2'] = data_2['Mã khu vực mua hàng C2'].astype('int64')

data_2['Mã siêu thị map nhu cau 911'] = data_2['Mã siêu thị']
data_2.loc[((data_2['Mode'].isin([0,2]))), 'Mã siêu thị map nhu cau 911'] = data_2['Mã kho nhận C1']


data_2 = data_2.merge(df911_nhucau, how='left', on=['date', 'Mã siêu thị map nhu cau 911', 'Mã Model', 'Mã kho nhận C2', 'Mã khu vực mua hàng C2'], suffixes=('', '_2'))


data_2.head()

In [50]:
columns_to_convert = ['nhucautheosb', 'nhucaust','Số lượng cầu ban đầu', 'Số lượng cần mua', 'Số lượng đang đặt', 'Số lượng đặt về kho trung tâm', 'Tồn kho', 'Tồn kho trung tâm','Số lượng sc được nhận', 'Số lượng sc đã chuyển đi', 'SL mapping','Số lượng hàng đã đặt']
data_2[columns_to_convert] = data_2[columns_to_convert].fillna(0)
data_2[columns_to_convert] = data_2[columns_to_convert].astype('int64')

data_2_g = data_2.groupby(['date', 'Mã Model', 'Mã siêu thị map nhu cau 911', 'Mã khu vực mua hàng C2'])['nhucaust'].sum().reset_index()
data_2_g.rename(columns={'nhucaust': 'nhucaust_by_Mã siêu thị map nhu cau 911'}, inplace=True)
data_2_g = data_2_g.merge(df911_nhucau[['date', 'Mã Model', 'Mã siêu thị map nhu cau 911', 'Mã khu vực mua hàng C2','Số lượng cần mua', 'Tồn kho', 'Số lượng đang đặt']], how='left', on=['date', 'Mã Model', 'Mã siêu thị map nhu cau 911', 'Mã khu vực mua hàng C2'])

data_2_g['SL thiếu (khi sum nhu cầu)'] = np.maximum(data_2_g['nhucaust_by_Mã siêu thị map nhu cau 911'] - (data_2_g['Số lượng cần mua']+data_2_g['Tồn kho'] + data_2_g['Số lượng đang đặt']), 0)

# Hàm tùy chỉnh để tính giá trị của cột mới
def calculate_new_column(row):
    if row['nhucaust_by_Mã siêu thị map nhu cau 911'] == 0:
        return 0
    else:
        return row['SL thiếu (khi sum nhu cầu)'] / row['nhucaust_by_Mã siêu thị map nhu cau 911']

# Áp dụng hàm tùy chỉnh để tạo cột mới
data_2_g['%SL thiếu (khi sum nhu cầu)_row'] = data_2_g.apply(calculate_new_column, axis=1)


data_2_g['SL thiếu (khi sum nhu cầu, không trừ đi đường)'] = np.maximum(data_2_g['nhucaust_by_Mã siêu thị map nhu cau 911'] - (data_2_g['Số lượng cần mua']+data_2_g['Tồn kho'] ), 0)

# Hàm tùy chỉnh để tính giá trị của cột mới
def calculate_new_column(row):
    if row['nhucaust_by_Mã siêu thị map nhu cau 911'] == 0:
        return 0
    else:
        return row['SL thiếu (khi sum nhu cầu, không trừ đi đường)'] / row['nhucaust_by_Mã siêu thị map nhu cau 911']

# Áp dụng hàm tùy chỉnh để tạo cột mới
data_2_g['%SL thiếu (khi sum nhu cầu, không trừ đi đường)_row'] = data_2_g.apply(calculate_new_column, axis=1)


data_2_g.drop(columns=['Số lượng cần mua', 'Tồn kho', 'Số lượng đang đặt'], inplace=True)
data_2 = data_2.merge(data_2_g, how='left', on=['date', 'Mã Model', 'Mã siêu thị map nhu cau 911', 'Mã khu vực mua hàng C2'])

data_2['SL thiếu (khi mapping)'] = np.maximum(data_2['Số lượng cần mua'] - data_2['SL mapping'],0)



# Hàm tùy chỉnh để tính giá trị của cột mới
def calculate_new_column(row):
    if row['Số lượng cần mua'] == 0:
        return 0
    else:
        return row['SL thiếu (khi mapping)'] / row['Số lượng cần mua']

# Áp dụng hàm tùy chỉnh để tạo cột mới
data_2['%SL thiếu (khi mapping)_row'] = data_2.apply(calculate_new_column, axis=1)




#### PO

In [51]:
df_89 = Read_feather_folder(r'D:\Lostsale ĐM\89\89ft')
df_89.drop_duplicates(inplace=True)
df_308 = Read_feather_folder(r'D:\Lostsale ĐM\308\308ft')
df_308.drop_duplicates(inplace=True)

In [ ]:
df_89['date'] = pd.to_datetime(df_89['Ngày đặt']).dt.date
df_89['Mã PO'] = df_89['Mã PO'].astype('str')
df_89['Mã PO'] = df_89['Mã PO'].str.strip()
df_89['Mã sản phẩm'] = df_89['Mã sản phẩm'].astype('str')

df_308['Mã PO'] = df_308['Mã PO'].astype('str')
df_308['Mã PO'] = df_308['Mã PO'].str.strip()
df_308['Mã Model'] = df_308['Mã Model'].astype('str')
df_308['Mã sản phẩm'] = df_308['Mã sản phẩm'].astype('str')
df_308['date'] = pd.to_datetime(df_308['Ngày đặt']).dt.date
df_308['fr_date'] = pd.to_datetime(df_308['Ngày nhập từ']).dt.date
df_308['to_date'] = pd.to_datetime(df_308['Ngày nhập đến']).dt.date
df_308 = df_308[(df_308['Nhân viên nhập'].isin(['60802 - Phan Kim Ngọc', '105697 - Lê Minh Đức', '120651 - Đào Thị Ngọc Ánh', 'administrator - Administrator']))]

df_308.loc[df_308['Tên KH/NCC'] == 'BHX KHO ĐÔNG', 'Sl còn lại'] = 0

df_308['Mã siêu thị nhận hàng'] = df_308['Tên siêu thị'].str.extract('(^[0-9]{0,10})').astype('int64')
df_308['Ngày gia hạn'] = df_308['Ngày gia hạn'].fillna(0).astype('str')
# df_308['Nội dung'] = df_308['Nội dung'].fillna(0).astype('str')
df_89 = df_89[['Mã siêu thị', 'Mã PO', 'Mã sản phẩm', 'Số lượng', 'date']].drop_duplicates()
df_308 = df_308[['date', 'fr_date', 'to_date', 'Mã siêu thị nhận hàng', 'Mã PO', 'Mã NCC', 'Tên KH/NCC', 'Mã khu vực mua hàng', 'Khu vực mua hàng', 'Tên hình thức nhập', 'Mã sản phẩm', 'Số lượng đặt', 'Sl còn lại', 'Tên loại đơn hàng', 'Ngày gia hạn']]
df_308_g = df_308.groupby(['date', 'fr_date', 'to_date', 'Mã siêu thị nhận hàng', 'Mã PO', 'Mã sản phẩm', 'Tên loại đơn hàng','Ngày gia hạn'])['Số lượng đặt', 'Sl còn lại'].sum().reset_index()

df_308_g.head()

In [53]:
df_308_g.rename(columns={'Mã sản phẩm': 'Mã sản phẩm cơ sở'}, inplace=True)
df_308_g['Mã sản phẩm cơ sở'] = df_308_g['Mã sản phẩm cơ sở'].astype('int64')
data_2['Mã sản phẩm cơ sở'] = data_2['Mã sản phẩm cơ sở'].astype('int64')

df_308_g['date'] = pd.to_datetime(df_308_g['date'])
data_2['Mã PO'] = data_2['Mã PO'].str.rstrip()
df_khu_vuc_308 = data_2.merge(df_308_g, how='left', on=['Mã PO', 'Mã sản phẩm cơ sở', 'date'])


data_2_g_PO = data_2[['Mã kho nhận C2', 'Mã sản phẩm cơ sở', 'date','SL mapping']].drop_duplicates().groupby(['Mã kho nhận C2','Mã sản phẩm cơ sở', 'date'])['SL mapping'].sum().reset_index()
data_2_g_PO.rename(columns={'SL mapping': 'SL mapping sum theo PO'}, inplace=True)
df_khu_vuc_308 = df_khu_vuc_308.merge(data_2_g_PO, on=['Mã kho nhận C2', 'Mã sản phẩm cơ sở', 'date'], how='left')
df_khu_vuc_308.head()

,Mã siêu thị,Mã Model,Tên siêu thị,date,Mã kho nhận C1,Mã khu vực mua hàng C1,Mã kho nhận C2,Mã khu vực mua hàng C2,no_data,Mã sản phẩm cơ sở,nhucautheosb,nhucaust,leadtime,fcfromdate,fctodate,CAPACITYRAW,Ngày nhận hàng,Tên sản phẩm,Mã chi nhánh,Tên chi nhánh NCC,Tên khu vực,Mã loại khu vực mua hàng,Mã kho nhận hàng,Tên kho nhận hàng,Hình thức giao hàng,Mã kho nhận hàng quá cảnh,Tên kho nhận hàng quá cảnh,Có hóa đơn về cùng đơn hàng,Khả năng cung ứng,Khả năng giao tối thiểu theo khu vực,Mã NCC,Điểm,% Chất lượng sản phẩm,Điểm Chất lượng,% Integrity SL giao,Điểm Integrity,% Giá tốt,Điểm giá tốt,Điểm hồ sơ QC mua/chia,NCC đứt hàng,Tên nhóm hàng,Lịch tạo PO theo thứ (thường),weekday,index,có lịch,Mode,Mã kho nhận C1 là hub,fcfromdate +1,Mã siêu thị map nhu cau 911,Số lượng cầu ban đầu,Số lượng cần mua,Số lượng đang đặt,Số lượng đặt về kho trung tâm,Tồn kho,Tồn kho trung tâm,Số lượng sc được nhận,Số lượng sc đã chuyển đi,SL mapping,Mã PO,Số lượng hàng đã đặt,Ghi chú vi phạm đơn hàng,Ghi chú vi phạm sản phẩm,SL vi phạm thể tích,nhucaust_by_Mã siêu thị map nhu cau 911,SL thiếu (khi sum nhu cầu),%SL thiếu (khi sum nhu cầu)_row,"SL thiếu (khi sum nhu cầu, không trừ đi đường)","%SL thiếu (khi sum nhu cầu, không trừ đi đường)_row",SL thiếu (khi mapping),%SL thiếu (khi mapping)_row,fr_date,to_date,Mã siêu thị nhận hàng,Tên loại đơn hàng,Ngày gia hạn,Số lượng đặt,Sl còn lại,SL mapping sum theo PO
0,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-12,22859,27926.0,6320,23901,0.0,9904639000030,0,0,3,2023-11-14,20231122,15,NaN,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,15711.0,BHX <> CÔNG TY CỔ PHẦN LOTHAMILK,HÀNG MÁT - SỮA_KHO TĐN (6320 chia DC Mini),193.0,6320.0,BHX_HCM_BCH - Kho Đông Mát Trần Đại Nghĩa (HCM),1.0,NaN,None,1.0,999999.0,0.0,27834.0,6.0,0%,3.0,0%,1.0,0%,1.0,1.0,0.0,Sữa Các Loại,"T2, T3, T4, T5, T6, T7",6,NaN,0.0,0,1.0,2023-11-15,22859,12,12,30,0,0,0,0,0,0,None,0,None,None,NaN,0,0,0.0,0,0.0,12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-11,22859,27926.0,6320,23901,0.0,9904639000030,0,0,3,2023-11-13,20231121,15,NaN,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,15711.0,BHX <> CÔNG TY CỔ PHẦN LOTHAMILK,HÀNG MÁT - SỮA_KHO TĐN (6320 chia DC Mini),193.0,6320.0,BHX_HCM_BCH - Kho Đông Mát Trần Đại Nghĩa (HCM),1.0,NaN,None,1.0,999999.0,0.0,27834.0,6.0,0%,3.0,0%,1.0,0%,1.0,1.0,0.0,Sữa Các Loại,"T2, T3, T4, T5, T6, T7",5,18297.0,1.0,0,1.0,2023-11-14,22859,12,12,54,0,0,0,0,0,12,06320PO2311762657,12,None,None,0.0,0,0,0.0,0,0.0,0,0.0,2023-11-12,2023-11-14,6320.0,PO thường,0,372.0,186.0,78
2,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-10,22859,27926.0,6320,23901,0.0,9904639000030,0,0,3,2023-11-12,20231120,15,NaN,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,15711.0,BHX <> CÔNG TY CỔ PHẦN LOTHAMILK,HÀNG MÁT - SỮA_KHO TĐN (6320 chia DC Mini),193.0,6320.0,BHX_HCM_BCH - Kho Đông Mát Trần Đại Nghĩa (HCM),1.0,NaN,None,1.0,999999.0,0.0,27834.0,6.0,0%,3.0,0%,1.0,0%,1.0,1.0,0.0,Sữa Các Loại,"T2, T3, T4, T5, T6, T7",4,18296.0,1.0,0,1.0,2023-11-13,22859,18,18,36,0,24,0,0,24,18,06320PO2311738810,18,None,None,0.0,0,0,0.0,0,0.0,0,0.0,2023-11-11,2023-11-12,6320.0,PO thường,11/10/2023 10:18:25 AM,360.0,0.0,102
3,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-09,22859,27926.0,6320,23901,0.0,9904639000030,0,0,3,2023-11-11,20231119,15,NaN,SỮA TƯƠI TRÂN CHÂU ĐƯỜNG ĐEN LOTHAMILK CHAI 25...,15711.0,BHX <> CÔNG TY CỔ PHẦN LOTHAMILK,HÀNG MÁT - SỮA_KHO TĐN (6320 chia DC Mini),193.0,6320.0,BHX_HCM_BCH - Kho Đông Mát Trần Đại Nghĩa (HCM),1.0,NaN,None,1.0,999999.0,0.0,27834.0,6.0,0%,3.0,0%,1.0,0%,1.0,1.0,0.0,Sữa Các Loại,"T2, T3, T4, T5, T6, T7",3,18295.0,1.0,0,1.0,2023-11-12,22859,36,36,24,0,90,0,0,90,36,06320PO2311718176,36,None,None,0.0,0,0,0.0,0,0.0,0,0.0,2023-11-10,2023-11-11,6320.0,PO thường,0,660.0,0.0,150
4,27921,2301000763,BHX_HCM_GVA - Cityland Park Hills,2023-11-08,22859,27926.0,6320,23901,0.0,9904639000030,0,0,3,2023-11-10,20231118,15,NaN,SỮA TƯƠI TRÂN

In [ ]:
df_89.rename(columns={'Mã siêu thị': 'Mã kho nhận hàng C2', 'Mã sản phẩm': 'Mã sản phẩm cơ sở'}, inplace=True)
df_89['date'] = pd.to_datetime(df_89['date'])
df_89['Mã sản phẩm cơ sở'] = df_89['Mã sản phẩm cơ sở'].astype('int64')
df_khu_vuc_308['Mã kho nhận hàng C2'] = df_khu_vuc_308.apply(lambda row: row['Mã siêu thị'] if row['Mã kho nhận C1'] == row['Mã kho nhận C2'] else row['Mã kho nhận C1'], axis=1)
df_khu_vuc_308_89 = df_khu_vuc_308.merge(df_89, how='left', on=['Mã PO', 'Mã kho nhận hàng C2', 'date', 'Mã sản phẩm cơ sở'])
df_khu_vuc_308_89.head()



#### Chia hang

##### 1. Kho TT

In [ ]:
df_531 = Read_feather_folder(r'D:\Lostsale ĐM\531\531 ft')
df_531.drop_duplicates(inplace=True)
df_531 = df_531[['Ngày nhập đến', 'Mã siêu thị', 'Mã sản phẩm', 'Kho chuyển', 'Danh sách PO', 'Đơn vị', 'Tổng số lượng', 'Số lượng hàng đã đặt', 'SL Ký/Gói', 'rescaledate']]
df_531['rescaledate'] = pd.to_datetime(df_531['rescaledate']).dt.date
df_531['rescaledate'] = pd.to_datetime(df_531['rescaledate'])
df_531['Ngày nhập đến'] = pd.to_datetime(df_531['Ngày nhập đến'])
df_531.head()

In [ ]:
df1 = df_531[['Danh sách PO']].drop_duplicates()
df2 = df1.assign(**{'Mã PO': df1['Danh sách PO'].str.split(',')}).explode('Mã PO').reset_index(drop=True)
df2[df2['Danh sách PO'] == '08192PO2309457250,08192PO2309461583']
df2.drop_duplicates(inplace=True)

df_531 =  df_531.merge(df2, how='left', on='Danh sách PO')
df_531.head()

In [57]:
df_531 = df_531.drop_duplicates()
df_531.drop(columns=['Ngày nhập đến', 'Kho chuyển', 'Danh sách PO', 'Đơn vị', 'Tổng số lượng'], inplace=True)


In [58]:
df_531.rename(columns={'Mã sản phẩm': 'Mã sản phẩm cơ sở', 'Mã siêu thị': 'Mã kho nhận hàng C2', 'Số lượng hàng đã đặt': 'Số lượng hàng đã đặt 531'}, inplace=True)
data = df_khu_vuc_308_89.merge(df_531, how='left', on=['Mã PO', 'Mã sản phẩm cơ sở', 'Mã kho nhận hàng C2'])
data['Ngày gia hạn'] = data['Ngày gia hạn'].fillna(0).astype('str')
# data['Nội dung'] = data['Nội dung'].fillna(0).astype('str')



In [59]:

data.rename(columns={'Số lượng': 'Số lượng 89'}, inplace=True)

In [60]:
df_308_g_POthucong = df_308_g[df_308_g['Tên loại đơn hàng'] == 'PO thủ công']
df_89['Mã PO'] = df_89['Mã PO'].str.strip()
df_308_g_POthucong_89 = df_308_g_POthucong.merge(df_89, how='left', on=['date', 'Mã PO', 'Mã sản phẩm cơ sở'])
df_308_g_POthucong_89_531 = df_308_g_POthucong_89.merge(df_531, how='left', on=['Mã kho nhận hàng C2', 'Mã sản phẩm cơ sở', 'Mã PO'])
df_308_g_POthucong_89_531.groupby(['date', 'Mã kho nhận hàng C2', 'Mã sản phẩm cơ sở'])['Mã PO'].count().sort_values(ascending=False)

df_308_g_POthucong_89_531.head()

,date,fr_date,to_date,Mã siêu thị nhận hàng,Mã PO,Mã sản phẩm cơ sở,Tên loại đơn hàng,Ngày gia hạn,Số lượng đặt,Sl còn lại,Mã kho nhận hàng C2,Số lượng,Số lượng hàng đã đặt 531,SL Ký/Gói,rescaledate
0,2023-11-01,2023-11-01,2023-11-06,3615,03615PO2311588688,9902980000023,PO thủ công,0,80,0,3615.0,40.0,NaN,NaN,NaT
1,2023-11-01,2023-11-01,2023-11-06,3616,03616PO2311588689,9902980000023,PO thủ công,0,180,0,3616.0,90.0,NaN,NaN,NaT
2,2023-11-01,2023-11-01,2023-11-06,3650,03650PO2311588690,9902980000023,PO thủ công,0,20,0,3650.0,10.0,NaN,NaN,NaT
3,2023-11-01,2023-11-01,2023-11-06,3682,03682PO2311588691,9902980000023,PO thủ công,0,20,0,3682.0,10.0,NaN,NaN,NaT
4,2023-11-01,2023-11-01,2023-11-06,3683,03683PO2311588692,9902980000023,PO thủ công,0,40,0,3683.0,20.0,NaN,NaN,NaT


In [61]:
df_308_g_POthucong_89_531_g = df_308_g_POthucong_89_531.groupby(['date', 'Mã kho nhận hàng C2', 'Mã sản phẩm cơ sở'])['Số lượng đặt', 'Sl còn lại', 'Số lượng', 'Số lượng hàng đã đặt 531', 'SL Ký/Gói', 'to_date'].agg({'Số lượng đặt': 'sum', 'Sl còn lại': 'sum', 'Số lượng': 'sum', 'Số lượng hàng đã đặt 531': 'sum', 'SL Ký/Gói': 'sum', 'to_date': 'max'}).reset_index()
df_308_g_POthucong_89_531_g.head()

,date,Mã kho nhận hàng C2,Mã sản phẩm cơ sở,Số lượng đặt,Sl còn lại,Số lượng,Số lượng hàng đã đặt 531,SL Ký/Gói,to_date
0,2023-11-01,191.0,8935217413201,3672,192,54.0,0,0,2023-11-03
1,2023-11-01,1374.0,8935217413201,3672,192,54.0,0,0,2023-11-03
2,2023-11-01,1521.0,1012841000062,128,0,8.0,0,0,2023-11-03
3,2023-11-01,1630.0,1353926000001,16,0,4.0,0,0,2023-11-03
4,2023-11-01,1630.0,9904639000030,36,36,6.0,0,0,2023-11-03


In [62]:
data.drop_duplicates(inplace=True)
data = data.merge(df_308_g_POthucong_89_531_g, how='left', on=['date', 'Mã kho nhận hàng C2', 'Mã sản phẩm cơ sở'], suffixes=('',' PO thucong'))

In [63]:

data = data[(data['fcfromdate +1']>=(pd.to_datetime(today) - timedelta(days=7))) & (data['fcfromdate +1']<(pd.to_datetime(today)))]

## Calculation

In [64]:
# data.drop(columns=['level_0', 'index'], inplace=True)
data.sort_values(by=['Mã siêu thị', 'Mã Model', 'date'], inplace=True)
data.drop_duplicates(subset=['date','Mã siêu thị','Mã Model'], inplace=True)
data.reset_index(inplace=True)


df_all_cd.sort_values(by=['Mã siêu thị', 'Mã Model', 'date'], inplace=True)
df_all_cd.drop_duplicates(subset=['date','Mã siêu thị','Mã Model'], inplace=True)
df_all_cd.reset_index(inplace=True)


In [65]:
#1. Sum xuất hàng  7 ngày trước theo từng hình thức
df_all_cd.fillna(0, inplace=True)
df_all_cd['Nhập'] = df_all_cd["('Số lượng im', 'Nhập chuyển kho')"]   + df_all_cd["('Số lượng im', 'Nhập mua nội bộ')"] 
df_all_cd['Bán'] =  -df_all_cd["('Số lượng im', 'Nhập trả hàng BHX Online Pick tại Siêu Thị')"] -df_all_cd["('Số lượng im', 'Nhập trả lại của hàng bán tại siêu thị')"] -df_all_cd["('Số lượng im', 'Nhập trả lại của hàng xuất khuyến mãi')"] + df_all_cd["('Số lượng out', 'Xuất bán hàng BHX Online Pick tại Siêu Thị')"] + df_all_cd["('Số lượng out', 'Xuất bán hàng tại siêu thị')"] 
df_all_cd['Xuất chuyển tồn'] = df_all_cd["('Số lượng out', 'Xuất chuyển kho')"] + df_all_cd["('Số lượng out', 'Xuất bán nội bộ')"]
df_all_cd['Xuất kiểm kê'] = -df_all_cd["('Số lượng im', 'Nhập thừa so kiểm kê với sổ sách')"] + df_all_cd["('Số lượng out', 'Xuất thiếu so kiểm kê với sổ sách')"]
df_all_cd['Tặng/Bán cận date/hư hỏng'] = df_all_cd["Sl bán cận date"] + df_all_cd["Số lượng tặng do cận date/hư hỏng"]
 

df_all_cd.sort_values(by=['Mã siêu thị', 'Mã Model', 'date'], inplace=True)
df_all_cd['CumulativeSum_Nhập'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])['Nhập'].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
df_all_cd['CumulativeSum_Bán'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])['Bán'].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
df_all_cd['CumulativeSum_Xuất chuyển tồn'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])['Xuất chuyển tồn'].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
df_all_cd['CumulativeSum_Xuất kiểm kê'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])['Xuất kiểm kê'].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
df_all_cd['CumulativeSum_Tặng/Bán cận date/hư hỏng'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])['Tặng/Bán cận date/hư hỏng'].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
df_all_cd['CumulativeSum_Hủy'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])["('Số lượng out', 'Xuất hủy')"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
df_all_cd['CumulativeSum_Xuất khuyến mãi'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])["('Số lượng out', 'Xuất khuyến mãi')"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
#df_all_cd['CumulativeSum_Xuất khác'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])['Xuất khác'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
df_all_cd['CumulativeSum_Tổng xuất'] = df_all_cd['CumulativeSum_Bán'] + df_all_cd['CumulativeSum_Xuất chuyển tồn'] + df_all_cd['CumulativeSum_Xuất kiểm kê']  + df_all_cd['CumulativeSum_Hủy'] + df_all_cd['CumulativeSum_Xuất khuyến mãi'] 


In [ ]:
# 2. Count số ngày có tồn hoặc có bán trong khoảng thời gian đó
df_all_cd['Có Xuất'] = np.minimum(df_all_cd["('Số lượng out', 'Xuất bán hàng BHX Online Pick tại Siêu Thị')"]  + df_all_cd["('Số lượng out', 'Xuất bán hàng tại siêu thị')"] + df_all_cd["('Số lượng out', 'Xuất chuyển kho')"] + df_all_cd["('Số lượng out', 'Xuất hủy')"] ,1)
df_all_cd['Có Tồn'] = np.minimum(df_all_cd["Tồn thực tế ở ST"],1)
stock_count = df_all_cd[df_all_cd['Có Tồn'] == 1][['date', 'Mã siêu thị', 'Mã Model']].drop_duplicates()
out_count = df_all_cd[df_all_cd['Có Xuất'] == 1][['date', 'Mã siêu thị', 'Mã Model']].drop_duplicates()
count = stock_count.merge(out_count, how='outer', on=['date', 'Mã siêu thị', 'Mã Model'])
count['Có xuất hoặc có tồn'] = 1

df_all_cd = df_all_cd.merge(count,  on=['date', 'Mã siêu thị', 'Mã Model'], how='left')
df_all_cd['Có xuất hoặc có tồn'] = df_all_cd['Có xuất hoặc có tồn'].fillna(0)
df_all_cd['CumulativeSum_Có xuất hoặc có tồn'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])["Có xuất hoặc có tồn"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
df_all_cd.head()

In [67]:
# 4. Sức bán trung bình (Hệ thống)
df_all_cd['CumulativeMean_Sức bán_347'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])["Sức bán"].rolling(window=7, min_periods=1).mean().reset_index(drop=True)

In [68]:
# 5. Tổng nhu cầu theo SB những ngày có lịch

# data['nhucautheosb'] = data['nhucautheosb'].fillna(0)
# data['CumulativeSum_nhucautheosb'] = data.groupby(['Mã siêu thị', 'Mã Model']).apply(lambda x: x.loc[x['có lịch'] == 1, 'nhucautheosb'].rolling(window=7, min_periods=1).sum()).reset_index(drop=True)
# data['CumulativeSum_nhucautheosb'] = data['CumulativeSum_nhucautheosb'].fillna(method='ffill')


data['có lịch'] = data['có lịch'].astype('int64')
data['nhucautheosb_lich'] = data['có lịch']* data['nhucautheosb']
data['CumulativeSum_nhucautheosb'] = data.groupby(['Mã siêu thị', 'Mã Model'])['nhucautheosb_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='nhucautheosb_lich', inplace=True)

In [69]:
# 6. Tổng nhu cầu st những ngày có lịch
data['nhucaust'] = data['nhucaust'].fillna(0)
# data['CumulativeSum_nhucaust'] = data.groupby(['Mã siêu thị', 'Mã Model']).apply(lambda x: x.loc[x['có lịch'] == 1, 'nhucaust'].rolling(window=7, min_periods=1).sum()).reset_index(drop=True)
# data['CumulativeSum_nhucaust'] = data['CumulativeSum_nhucaust'].fillna(method='ffill')


data['nhucaust_lich'] = data['có lịch']* data['nhucaust']
data['CumulativeSum_nhucaust'] = data.groupby(['Mã siêu thị', 'Mã Model'])['nhucaust_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='nhucaust_lich', inplace=True)

In [70]:
# 7. Capacity trung bình  
data['CumulativeMean_CAPACITYRAW'] = data.groupby(['Mã siêu thị', 'Mã Model'])["CAPACITYRAW"].rolling(window=7, min_periods=1).mean().reset_index(drop=True)

In [71]:
# 8. Q/C mua trung bình 
df_all_cd['CumulativeMean_Quy cách mua'] = df_all_cd.groupby(['Mã siêu thị', 'Mã Model'])["Quy cách mua"].rolling(window=7, min_periods=1).mean().reset_index(drop=True)

In [72]:
# 9. Số ngày có lịch trong 1 tuần trước
data['CumulativeSum_Có lịch'] = data.groupby(['Mã siêu thị', 'Mã Model'])["có lịch"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 

In [73]:
# 10. SUM SL thiếu (khi sum nhu cầu) - SUm SL lên NC
data['SL thiếu (khi sum nhu cầu)'] = data['SL thiếu (khi sum nhu cầu)'].fillna(0)
data['SL thiếu (khi sum nhu cầu, không trừ đi đường)'] = data['SL thiếu (khi sum nhu cầu, không trừ đi đường)'].fillna(0)
data['nhucaust_by_Mã siêu thị map nhu cau 911'] = data['nhucaust_by_Mã siêu thị map nhu cau 911'].fillna(0)


data['SL thiếu (khi sum nhu cầu)_lich'] = data['có lịch']* data['SL thiếu (khi sum nhu cầu)']
data['CumulativeSum_SL thiếu (khi sum nhu cầu)'] = data.groupby(['Mã siêu thị', 'Mã Model'])['SL thiếu (khi sum nhu cầu)_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='SL thiếu (khi sum nhu cầu)_lich', inplace=True)

data['SL thiếu (khi sum nhu cầu, không trừ đi đường)_lich'] = data['có lịch']* data['SL thiếu (khi sum nhu cầu, không trừ đi đường)']
data['CumulativeSum_SL thiếu (khi sum nhu cầu, không trừ đi đường)'] = data.groupby(['Mã siêu thị', 'Mã Model'])['SL thiếu (khi sum nhu cầu, không trừ đi đường)_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='SL thiếu (khi sum nhu cầu, không trừ đi đường)_lich', inplace=True)

data['nhucaust_by_Mã siêu thị map nhu cau 911_lich'] = data['có lịch']* data['nhucaust_by_Mã siêu thị map nhu cau 911']
data['CumulativeSum_nhucaust_by_Mã siêu thị map nhu cau 911'] = data.groupby(['Mã siêu thị', 'Mã Model'])['nhucaust_by_Mã siêu thị map nhu cau 911_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='nhucaust_by_Mã siêu thị map nhu cau 911_lich', inplace=True)

data['% thiếu khi sum nhu cầu'] = data['CumulativeSum_SL thiếu (khi sum nhu cầu)'] / data['CumulativeSum_nhucaust_by_Mã siêu thị map nhu cau 911']
data['% thiếu khi sum nhu cầu (không trừ đi đường)'] = data['CumulativeSum_SL thiếu (khi sum nhu cầu, không trừ đi đường)'] / data['CumulativeSum_nhucaust_by_Mã siêu thị map nhu cau 911']

In [74]:
# 11. SUM SL thiếu (khi mapping) - SUM SL mapping
data['SL thiếu (khi mapping)'] = data['SL thiếu (khi mapping)'].fillna(0)
data['Số lượng cần mua'] = data['Số lượng cần mua'].fillna(0)
# data['CumulativeSum_SL thiếu (khi mapping)'] = data.groupby(['Mã siêu thị', 'Mã Model']).apply(lambda x: x.loc[x['có lịch'] == 1, 'SL thiếu (khi mapping)'].rolling(window=7, min_periods=1).sum()).reset_index(drop=True)
# data['CumulativeSum_SL thiếu (khi mapping)'] = data['CumulativeSum_SL thiếu (khi mapping)'].fillna(method='ffill') 

data['SL thiếu (khi mapping)_lich'] = data['có lịch']* data['SL thiếu (khi mapping)']
data['CumulativeSum_SL thiếu (khi mapping)'] = data.groupby(['Mã siêu thị', 'Mã Model'])['SL thiếu (khi mapping)_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='SL thiếu (khi mapping)_lich', inplace=True)




data['Số lượng cần mua_lich'] = data['có lịch']* data['Số lượng cần mua']
data['CumulativeSum_Số lượng cần mua'] = data.groupby(['Mã siêu thị', 'Mã Model'])['Số lượng cần mua_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='Số lượng cần mua_lich', inplace=True)

data['% thiếu khi mapping'] = data['CumulativeSum_SL thiếu (khi mapping)'] / data['CumulativeSum_Số lượng cần mua']

In [75]:
# 12. SUm SL thiếu (khi lên PO so với tự động) - SL PO
data['Số lượng đặt'] = data['Số lượng đặt'].fillna(0)
data['Số lượng đặt PO thucong'] = data['Số lượng đặt PO thucong'].fillna(0)
data['SL mapping sum theo PO'] = data['SL mapping sum theo PO'].fillna(0)

data['Tổng SL lên PO'] = data['Số lượng đặt'] + data['Số lượng đặt PO thucong']
data['Tổng SL thiếu (khi lên PO)'] = np.maximum(data['SL mapping sum theo PO'] - data['Tổng SL lên PO'],0)



# Hàm tùy chỉnh để tính giá trị của cột mới
def calculate_new_column(row):
    if row['SL mapping sum theo PO'] == 0:
        return 0
    else:
        return row['Tổng SL thiếu (khi lên PO)'] / row['SL mapping sum theo PO']

# Áp dụng hàm tùy chỉnh để tạo cột mới
data['%Tổng SL thiếu (khi lên PO)_row'] = data.apply(calculate_new_column, axis=1)



data['Tổng SL thiếu (khi lên PO)_lich'] = data['có lịch']* data['Tổng SL thiếu (khi lên PO)']
data['CumulativeSum_Tổng SL thiếu (khi lên PO)'] = data.groupby(['Mã siêu thị', 'Mã Model'])['Tổng SL thiếu (khi lên PO)_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='Tổng SL thiếu (khi lên PO)_lich', inplace=True)

data['SL mapping sum theo PO_lich'] = data['có lịch']* data['SL mapping sum theo PO']
data['CumulativeSum_SL mapping sum theo PO'] = data.groupby(['Mã siêu thị', 'Mã Model'])['SL mapping sum theo PO_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='SL mapping sum theo PO_lich', inplace=True)

data['% Tổng SL thiếu (khi lên PO)'] = data['CumulativeSum_Tổng SL thiếu (khi lên PO)']/data['CumulativeSum_SL mapping sum theo PO']

In [76]:
# 13. Concat Ghi chú vi phạm đơn hàng + Ghi chú vi phạm sản phẩm - date, vi phạm thể tích


data['Ghi chú vi phạm đơn hàng'] = data['Ghi chú vi phạm đơn hàng'].fillna('0')
data['Ghi chú vi phạm đơn hàng'] = data['Ghi chú vi phạm đơn hàng'].str.lower()

data['Ghi chú vi phạm sản phẩm'] = data['Ghi chú vi phạm sản phẩm'].fillna('0')
data['Ghi chú vi phạm sản phẩm'] = data['Ghi chú vi phạm sản phẩm'].str.lower()

data['Minorder'] = 0  
data.loc[(data['Ghi chú vi phạm đơn hàng'].str.contains('không đủ', case=False, na=False)) | (data['Ghi chú vi phạm sản phẩm'].str.contains('không đủ', case=False, na=False)), 'Minorder'] = 1

data['Khai báo'] = 0  
data.loc[data['Ghi chú vi phạm đơn hàng'].str.contains('không tồn tại', case=False, na=False), 'Khai báo'] = 1

data['Tối ưu lấy khuyến mãi'] = 0  
data.loc[(data['Ghi chú vi phạm sản phẩm'].str.contains('khuyến mãi', case=False, na=False)), 'Tối ưu khuyến mãi'] = 1

data['Khác'] = 0  
data.loc[((~data['Ghi chú vi phạm đơn hàng'].str.contains('không đủ', case=False, na=False)) & (~data['Ghi chú vi phạm đơn hàng'].str.contains('không tồn tại', case=False, na=False)) & (data['Ghi chú vi phạm đơn hàng']!='0')) | ((~data['Ghi chú vi phạm sản phẩm'].str.contains('không đủ', case=False, na=False)) & (~data['Ghi chú vi phạm sản phẩm'].str.contains('khuyến mãi', case=False, na=False)) & (data['Ghi chú vi phạm sản phẩm']!='0')), 'Khác'] = 1



data['SL vi phạm thể tích'] = data['SL vi phạm thể tích'].fillna(0)
data['Không vi phạm thể tích'] = np.where(data['SL vi phạm thể tích']==0,1,0)
data['Có vi phạm thể tích'] = np.where(data['SL vi phạm thể tích']>0,1,0)


data['Minorder'] = data['Minorder']* data['Không vi phạm thể tích'] 
data['Tối ưu lấy khuyến mãi'] = data['Tối ưu lấy khuyến mãi']* data['Không vi phạm thể tích'] 

data['CumulativeSum_Minorder'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Minorder"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
data['CumulativeSum_Tối ưu lấy khuyến mãi'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Tối ưu lấy khuyến mãi"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
data['CumulativeSum_Có vi phạm thể tích'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Có vi phạm thể tích"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
data['CumulativeSum_Khai báo'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Khai báo"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
data['CumulativeSum_Khác'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Khác"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 


data['SL mapping'] = data['SL mapping'].fillna(0)
data['sum_mapping'] = data.groupby(['Mã siêu thị', 'Mã Model'])["SL mapping"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
data['CumulativeSum_SL vi phạm thể tích'] = data.groupby(['Mã siêu thị', 'Mã Model'])["SL vi phạm thể tích"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 

# Hàm tùy chỉnh để tính giá trị của cột mới
def calculate_new_column(row):
    if row['sum_mapping'] == 0:
        return 0
    else:
        return row['CumulativeSum_SL vi phạm thể tích'] / row['sum_mapping']

# Áp dụng hàm tùy chỉnh để tạo cột mới
data['%SL vi phạm thể tích so với mapping'] = data.apply(calculate_new_column, axis=1)

In [77]:
# 14. SUM SL thiếu NCC giao hàng  với now> ngày nhập đến  - SL PO
data['PO đã hết hạn'] = 0
data['to_date_fake'] = data['to_date'].fillna(today + timedelta(days=1))
data['to_date_fake'] = pd.to_datetime(data['to_date_fake'])
today = pd.to_datetime(today)
data.loc[(data['to_date_fake'] - today).dt.days < 0, 'PO đã hết hạn'] = 1

data['PO thucong đã hết hạn'] = 0
data['to_date_fake'] = data['to_date PO thucong'].fillna(today + timedelta(days=1))

data['to_date_fake'] = pd.to_datetime(data['to_date_fake'])
today = pd.to_datetime(today)

data.loc[(data['to_date_fake'] - today).dt.days < 0, 'PO thucong đã hết hạn'] = 1

data.drop(columns='to_date_fake', inplace=True)

data['Sl còn lại'] = data['Sl còn lại'].fillna(0)
data['Sl còn lại PO thucong'] = data['Sl còn lại PO thucong'].fillna(0)
data['Số lượng đặt'] = data['Số lượng đặt'].fillna(0)
data['Số lượng đặt PO thucong'] = data['Số lượng đặt PO thucong'].fillna(0)

data['SL PO hết hạn giao thiếu'] = data['Sl còn lại'] * data['PO đã hết hạn'] + data['Sl còn lại PO thucong'] * data['PO thucong đã hết hạn']

data['SL lên PO hết hạn'] = data['Số lượng đặt'] * data['PO đã hết hạn'] + data['Số lượng đặt PO thucong'] * data['PO thucong đã hết hạn']



# Hàm tùy chỉnh để tính giá trị của cột mới
def calculate_new_column(row):
    if row['SL lên PO hết hạn'] == 0:
        return 0
    else:
        return row['SL PO hết hạn giao thiếu'] / row['SL lên PO hết hạn']

# Áp dụng hàm tùy chỉnh để tạo cột mới
data['%SL PO hết hạn giao thiếu_row'] = data.apply(calculate_new_column, axis=1)



data['CumulativeSum_SL PO hết hạn giao thiếu'] = data.groupby(['Mã siêu thị', 'Mã Model'])["SL PO hết hạn giao thiếu"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
data['CumulativeSum_SL lên PO hết hạn'] = data.groupby(['Mã siêu thị', 'Mã Model'])["SL lên PO hết hạn"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 
data['% giao thiếu'] = data['CumulativeSum_SL PO hết hạn giao thiếu'] / data['CumulativeSum_SL lên PO hết hạn']

In [78]:
# 15. SL 89
data['Số lượng 89'] = data['Số lượng 89'].fillna(0)
data['Số lượng'] = data['Số lượng'].fillna(0)

data['Tổng số lượng đã đặt 89'] = data['Số lượng 89'] + data['Số lượng']
data['CumulativeSum_SL 89'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Tổng số lượng đã đặt 89"].rolling(window=7, min_periods=1).sum().reset_index(drop=True) 

In [79]:
# 16. Tổng SL mapping những ngày có lịch

data['SL mapping'] = data['SL mapping'].fillna(0)
data['có lịch'] = data['có lịch'].astype('int64')
data['SL mapping_lich'] = data['có lịch']* data['SL mapping']
data['CumulativeSum_SL mapping'] = data.groupby(['Mã siêu thị', 'Mã Model'])['SL mapping_lich'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)
data.drop(columns='SL mapping_lich', inplace=True)

data['CumulativeSum_mapping trừ 89'] =np.maximum(data['CumulativeSum_SL mapping'] - data['CumulativeSum_SL 89'],0)

In [80]:
# 16. SL PO gia hạn / Tổng  số PO
data['Mã PO'] = data['Mã PO'].fillna('0')

data['Là PO gia hạn'] = 0
data.loc[data['Ngày gia hạn'] != '0', 'Là PO gia hạn'] = 1
data['CumulativeSum_SL PO gia hạn'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Là PO gia hạn"].rolling(window=7, min_periods=1).sum().reset_index(drop=True)  
data['Là PO'] = 1
data.loc[data['Mã PO'] == '0', 'Là PO'] = 0
data['CumulativeSum_SL PO'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Là PO"].rolling(window=7, min_periods=1).sum().reset_index(drop=True)  

data['% SL PO gia hạn'] = data['CumulativeSum_SL PO gia hạn'] / data['CumulativeSum_SL PO']


In [81]:
# 17. Tổng SL 531 - Tổng SL ký gói
data['Số lượng hàng đã đặt 531'] = data['Số lượng hàng đã đặt 531'].fillna(0)
data['Số lượng hàng đã đặt 531 PO thucong'] = data['Số lượng hàng đã đặt 531 PO thucong'].fillna(0)
data['SL Ký/Gói'] = data['SL Ký/Gói'].fillna(0)
data['SL Ký/Gói PO thucong'] = data['SL Ký/Gói PO thucong'].fillna(0)

data['Tổng SL 531'] = data['Số lượng hàng đã đặt 531'] +  data['Số lượng hàng đã đặt 531 PO thucong']
data['Tổng SL nhận 531'] = data['SL Ký/Gói'] +  data['SL Ký/Gói PO thucong']
data['CumulativeSum_Tổng SL 531'] = data.groupby(['Mã siêu thị', 'Mã Model'])["Tổng SL 531"].rolling(window=7, min_periods=1).sum().reset_index(drop=True)  
data['CumulativeSum_Tổng SL nhận 531'] = data.groupby(['Mã siêu thị', 'Mã Model', 'date'])["Tổng SL nhận 531"].rolling(window=7, min_periods=1).sum().reset_index(drop=True)  

data['% nhận/đặt 531'] = data['CumulativeSum_Tổng SL nhận 531'] / data['CumulativeSum_Tổng SL 531']

In [ ]:
# 18 Ngày được nhập hàng gần nhất và Số lượng được nhập gần nhất
df_all_cd['Nhập'] = df_all_cd['Nhập'].fillna(0)
the_lastdate = df_all_cd[df_all_cd['Nhập'] > 0][['Mã siêu thị', 'Mã Model', 'date', 'Nhập']].drop_duplicates()
the_lastdate.sort_values(by=['date'], inplace=True)
the_lastdate.rename(columns={'date':'date_last'}, inplace=True)
df_all_cd.sort_values(by=['date'], inplace=True)
df_all_cd = pd.merge_asof(df_all_cd, the_lastdate, left_on='date',right_on='date_last', by=['Mã siêu thị', 'Mã Model'], direction='backward', suffixes=('', '_last'), allow_exact_matches=False)
df_all_cd.head()

In [83]:
# 19 Trừ hàng đi đường
# XĐ số mua của ngày có PO thì được bao nhiêu ngày

data['%SL thiếu (khi sum nhu cầu)_row']=  data['%SL thiếu (khi sum nhu cầu)_row'].fillna(0)
data['%SL thiếu (khi mapping)_row']=  data['%SL thiếu (khi mapping)_row'].fillna(0)
data['%Tổng SL thiếu (khi lên PO)_row']=  data['%Tổng SL thiếu (khi lên PO)_row'].fillna(0)
data['%SL PO hết hạn giao thiếu_row']=  data['%SL PO hết hạn giao thiếu_row'].fillna(0)
data['%SL thiếu (khi sum nhu cầu, không trừ đi đường)_row']=  data['%SL thiếu (khi sum nhu cầu, không trừ đi đường)_row'].fillna(0)

data['nhucaust_PO'] = data['nhucaust']*data['Là PO']*(1-data['%SL thiếu (khi sum nhu cầu)_row'])*(1-data['%SL thiếu (khi mapping)_row'])*(1-data['%Tổng SL thiếu (khi lên PO)_row'])

data['nhucaust_PO NCC giao'] = data['nhucaust_PO']*(1-data['%SL PO hết hạn giao thiếu_row'])


df_sb = df_all_cd[['date', 'Mã siêu thị', 'Mã Model', 'Sức bán']].drop_duplicates()
df_sb.rename(columns={'date': 'fcfromdate +1', 'Sức bán': 'Sức bán fcfromdate'}, inplace=True)
data = data.merge(df_sb, how='left', on=['Mã siêu thị', 'Mã Model', 'fcfromdate +1'])


List_sb = data.groupby(['Mã siêu thị', 'Mã Model'])['Sức bán fcfromdate'].apply(list).reset_index()

List_sb.rename(columns={'Sức bán fcfromdate': 'Sức bán fcfromdate array'}, inplace=True)
data = data.merge(List_sb, how='left', on=['Mã siêu thị', 'Mã Model'])

data['RN'] = data.sort_values(['Mã siêu thị', 'Mã Model', 'date']) \
             .groupby(['Mã siêu thị', 'Mã Model']) \
             .cumcount() 

In [84]:
def slice_list(lst, start_index):
    return lst[start_index:]


data['filtered_values'] = data.apply(lambda row: slice_list(row['Sức bán fcfromdate array'], row['RN']), axis=1)

In [85]:

def find_index(nhucaust,x):
    try:
        x_array = np.array(x)
        x_array = pd.Series(x_array)
        x_array = x_array.dropna()
        x_array_sum = x_array.cumsum().values
        result_array = nhucaust - x_array_sum
        index = np.argmin(result_array[result_array > 0])
        return index
    except ValueError:
        return None


In [86]:
data['index_no'] = data.apply(lambda row: find_index(row['nhucaust_PO'],row['filtered_values']),axis=1)
# Chuyển đổi cột 'DaysToAdd' thành kiểu timedelta
data['DaysToAdd'] = pd.to_timedelta(data['index_no'], unit='D')

# Cộng cột 'Date' với cột 'DaysToAdd' để tạo cột 'NewDate'
data['ngày bán đến'] = pd.to_datetime(data['fcfromdate +1']) + data['DaysToAdd']


data.drop(columns=['Sức bán fcfromdate', 'Sức bán fcfromdate array', 'RN', 'filtered_values', 'index_no'], inplace=True)


data['Ngày bán đến PO trước đó'] = data.groupby(['Mã siêu thị', 'Mã Model', 'Là PO'])['ngày bán đến'].shift().where(data['Là PO'] == 1, '')

In [87]:
data['no_overlapped'] = np.where(data['Ngày bán đến PO trước đó'].isnull(), np.nan, (data['Ngày bán đến PO trước đó'] - data['fcfromdate +1']).dt.days)
data['no_overlapped'] = np.where((data['no_overlapped']< 0), 1, 0)

data['Có trừ hàng đi đường'] = np.where((data['%SL thiếu (khi sum nhu cầu)_row'] - data['%SL thiếu (khi sum nhu cầu, không trừ đi đường)_row']) !=0,1,0)
data['Vi phạm hàng đi đường'] = data['Có trừ hàng đi đường']*data['no_overlapped']

In [88]:
#nhucaust_PO
data['nhucaust_PO'] = data['nhucaust_PO'].fillna(0)
data['CumulativeSum_nhucaust_PO'] = data.groupby(['Mã siêu thị', 'Mã Model'])['nhucaust_PO'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)


data['nhucaust_PO NCC giao'] = data['nhucaust_PO NCC giao'].fillna(0)
data['CumulativeSum_nhucaust_PO NCC giao'] = data.groupby(['Mã siêu thị', 'Mã Model'])['nhucaust_PO NCC giao'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)

In [89]:
# Vi phạm hàng đi đường
data['Vi phạm hàng đi đường'] = data['Vi phạm hàng đi đường'].fillna(0)
data['CumulativeSum_Vi phạm hàng đi đường'] = data.groupby(['Mã siêu thị', 'Mã Model'])['Vi phạm hàng đi đường'].rolling(window=7, min_periods=1).sum().reset_index(drop=True)

## Final

In [90]:
data_final = pd.concat([data_1, data])
data_final.drop(columns=['date', 'Tên siêu thị', 'Mã sản phẩm cơ sở'], inplace=True)
data_final = data_final.merge(df_all_cd, how='right', right_on=['date', 'Mã siêu thị', 'Mã Model'], left_on=['fcfromdate +1', 'Mã siêu thị', 'Mã Model'])

two = data_2[(data_2['fcfromdate +1']>=(pd.to_datetime(today) - timedelta(days=7))) & (data_2['fcfromdate +1']<(pd.to_datetime(today)))].groupby(['Mã siêu thị', 'Mã Model'])['date'].nunique().reset_index()
two.rename(columns={'date': 'số ngày 2'}, inplace=True)
data_final = data_final.merge(two, how='left', on=['Mã siêu thị', 'Mã Model'])
data_final['số ngày 2'] = data_final['số ngày 2'].fillna(0)



In [91]:
data_final['Thiếu khai báo'] = np.where((data_final['số ngày 2']<= 5) | (data_final['CumulativeSum_Khai báo']>0), 1, 0)


data_final['Nhu cầu theo sức bán'] = data_final['CumulativeSum_nhucautheosb'] / data_final['CumulativeSum_Có lịch']
data_final['Nhu cầu'] = data_final['CumulativeSum_nhucaust'] / data_final['CumulativeSum_Có lịch']
data_final['Max ô vs QC mua'] = data_final['CumulativeMean_CAPACITYRAW'] / data_final['CumulativeMean_Quy cách mua']

data_final['Nhu cầu lên PO'] = data_final['CumulativeSum_nhucaust_PO'] / data_final['CumulativeSum_Có lịch']
data_final['Nhu cầu lên PO được NCC giao'] = data_final['CumulativeSum_nhucaust_PO NCC giao'] / data_final['CumulativeSum_Có lịch']
data_final['Nhập'] = data_final['CumulativeSum_Nhập'] / data_final['CumulativeSum_Có lịch']
data_final['Khoảng cách lịch tối đa'] = np.ceil(7 / data_final['CumulativeSum_Có lịch'])


data_final['Sức xuất'] = np.where(data_final['CumulativeSum_Có xuất hoặc có tồn'] ==0,0,data_final['CumulativeSum_Tổng xuất']/data_final['CumulativeSum_Có xuất hoặc có tồn'])


data_final['% xuất bán/xuất'] = np.where((data_final['CumulativeSum_Tổng xuất'])==0,0,(data_final['CumulativeSum_Bán'] +data_final['CumulativeSum_Xuất khuyến mãi']) /(data_final['CumulativeSum_Tổng xuất']))

data_final['Sức bán thực tế'] = np.where(data_final['CumulativeSum_Có xuất hoặc có tồn'] ==0,0,(data_final['CumulativeSum_Bán'] +data_final['CumulativeSum_Xuất khuyến mãi'])/data_final['CumulativeSum_Có xuất hoặc có tồn'])

In [92]:
first_stock = data_final[data_final['date'] == data_final['date'].min()][['Mã siêu thị', 'Mã sản phẩm cơ sở', 'Tồn thực tế ở ST']].drop_duplicates()
first_stock.rename(columns={'Tồn thực tế ở ST': 'Tồn đầu chu kỳ'}, inplace=True)
data_final = data_final.merge(first_stock, how='left', on=['Mã siêu thị', 'Mã sản phẩm cơ sở'])

In [93]:
data_final = data_final.merge(outstock_processed, on=['Mã sản phẩm cơ sở', 'date', 'Mã khu vực mua hàng C2'], how='left')

data_final['đứt hàng'] = data_final['đứt hàng'].fillna(0)

In [94]:
data_final[data_final['date'] == today - timedelta(days=1)][['Mã siêu thị' , 'Mã sản phẩm cơ sở','Thiếu khai báo', 'đứt hàng', 'Điểm', 'Max ô vs QC mua', 'Nhu cầu theo sức bán', 'Nhu cầu','Nhu cầu lên PO' ,'Nhu cầu lên PO được NCC giao','Nhập', '% thiếu khi sum nhu cầu', '% thiếu khi sum nhu cầu (không trừ đi đường)', '% thiếu khi mapping', '% Tổng SL thiếu (khi lên PO)', 'CumulativeSum_Minorder', 'CumulativeSum_Tối ưu lấy khuyến mãi', '% giao thiếu', 'CumulativeSum_SL PO', '% SL PO gia hạn', 'CumulativeSum_Có xuất hoặc có tồn','Tồn đầu chu kỳ', 'CumulativeSum_Nhập', 'CumulativeSum_Tổng xuất', '% xuất bán/xuất',  'Sức xuất', 'CumulativeMean_Sức bán_347','Sức bán thực tế', 'Khoảng cách lịch tối đa', 'CumulativeSum_Vi phạm hàng đi đường', 'CumulativeSum_Có lịch', 'CumulativeSum_Có vi phạm thể tích', '%SL vi phạm thể tích so với mapping', 'CumulativeSum_Khai báo', 'CumulativeSum_Khác']].to_parquet(r'D:\Lostsale ĐM\power_bi\ls_check.parquet')


In [ ]:
data_final.to_parquet(r'D:\Lostsale ĐM\power_bi\ls_check_all.parquet')

#### Volumn

In [95]:
# folder_path = r'D:\Lostsale ĐM\volumn\volumn dl'  # Replace with the path to your folder

# # Get a list of all files in the folder
# file_list = os.listdir(folder_path)

# # Filter the list to include only Excel files
# excel_files = [file for file in file_list if file.endswith('.xlsx') or file.endswith('.xls')]

# # Initialize an empty list to store individual DataFrames
# dfs = []

# # Read each Excel file and append its DataFrame to the list
# for file in excel_files:
#     file_path = os.path.join(folder_path, file)
#     df = pd.read_excel(file_path)
#     df['file_name'] = file
#     df.rename(columns={'m3 tối đa của kho': 'm3 tối đa', 'Kho': 'Thể tích chứa tối đa(m3) - Kho', 'Unnamed: 7': 'Thể tích chiếm giữ(m3) (bao gồm hàng đi đường) - Kho', 'Unnamed: 8': 'Thể tích chiếm giữ thực(m3) - Kho', 'Unnamed: 9': 'Thể tích còn lại(m3) - Kho', 'Trưng bày': 'Thể tích chứa tối đa(m3) - Trưng bày', 'Unnamed: 11': 'Thể tích khai báo Max ô(m3) - Trưng bày', 'Unnamed: 12': 'Thể tích chiếm giữ(m3) (bao gồm hàng đi đường) - Trưng bày', 'Unnamed: 13': 'Thể tích chiếm giữ thực(m3) - Trưng bày', 'Unnamed: 14': 'Thể tích còn lại(m3) - Trưng bày'}, inplace=True)
#     df.drop(df.index[0], inplace=True)
#     dfs.append(df)

# # Concatenate all DataFrames into a single DataFrame
# volumn = pd.concat(dfs, ignore_index=True)


In [96]:
# volumn['date'] =volumn['file_name'].astype('str').str[20:28]
# volumn['date'] = pd.to_datetime(volumn['date'], format='%d%m%Y')

# volumn['time'] =volumn['file_name'].astype('str').str[28:32]
# volumn['time'] = pd.to_datetime(volumn['time'], format='%H%M').dt.strftime('%H:%M')
# volumn.head()